In [ ]:
import os
import json
import pandas as pd
import re
from collections import defaultdict, Counter
from bs4 import BeautifulSoup

# 读取数据集
LABEL_PATH = os.path.join(os.getcwd()+'/')
label_trainset = json.load(open(LABEL_PATH + 'label_trainset.json'))
instruction_trainset = json.load(open(LABEL_PATH + 'instruction_trainset.json'))
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json'))

FileNotFoundError: ignored

In [ ]:
!pip install pandas

# 样本Prompt设计 （重要！！！）

1.全部网页元素（不告诉web) + instruction -> response（回答web+操作)  (len<512)

2.部分网页元素(不告诉web) + instruction -> response（回答web+操作） （len>512)   

3.web名称 + instruction  ->  response(操作，不回答web)。  （模型最终输入，输出）

In [ ]:
# 构建样本 prompt
'''
1.全部网页元素（不告诉web) + instruction -> response（回答web+操作) (len<512)
2.部分网页元素(不告诉web) + instruction -> response（回答web+操作）
3.web名称 + instruction  ->  response(操作，不回答web)
'''

import re

def encode_test_instruct(web_name, instruct, web_element=""):
    desc = "用户浏览的网页是[%s]，%s请回答执行什么操作能完成[%s]?" % (web_name, web_element, instruct)
    return desc

def encode_test_response(key_values):
    desc = "在网页上"
    for key, value in key_values.items():
        action = ""
        #print(key, value)
        if value["action"] == "点击":
            action = "[点击]名称为[%s]的[%s]" % (key, value["dom_type"])
        elif value["action"] == "输入":
            action = "[输入]值[%s]进名称为[%s]的[%s]" % (value["value"], key, value["dom_type"])
        desc += action + ";"
    return desc

def decode_response(desc):
    response = {}
    #actions = desc.split("。")[1]
    regex = r"(\[.*?\])"
    for action in desc.split(";"):
        matches = re.findall(regex, action)
        if len(matches) == 4:
            response[matches[2][1:-1]] = {'dom_type':matches[3][1:-1],
                                          'value':matches[1][1:-1],
                                          'action':'输入'}
        elif len(matches) == 3:
            response[matches[1][1:-1]] = {'dom_type':matches[2][1:-1],
                                          'value':'',
                                          'action':'点击'}
    return response


def trans_key_values2web_element(key_values):
    desc = ""
    for key, value in key_values.items():
        action = ""
        #print(key, value)
        if value["action"] == "点击":
            action = "可以[点击]名称为[%s]的[%s]" % (key, value["dom_type"])
        elif value["action"] == "输入":
            action = "可以[输入]名称为[%s]的[%s]" % (key, value["dom_type"])
        desc += action + "。"
    return desc

def encode_train_instruct1(instruct, web_element_all):  #实现构建样本中的情况1
    desc = "用户浏览网页包括的全部元素有:%s请先回答这是什么网页？再回答执行什么操作能完成[%s]？" % (web_element_all, instruct)
    return desc

def encode_train_instruct2(instruct, web_element_part): #实现构建样本中的情况2
    desc = "用户浏览网页包括的部分元素有:%s请先回答这是什么网页？再回答执行什么操作能完成[%s]？" % (web_element_part, instruct)
    return desc

def encode_train_response(web_name, key_value):
    desc = "用户浏览的网页是[%s]，" % web_name
    desc += encode_test_response(key_value)
    return desc

In [ ]:
instruct = "查询以下内容：会员编号为PT0100000022的机构在六年之内的信息"
web_name = "中国证券投资基金业协会"
key_value = {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
             '会员编号': {'dom_type': '输入框', 'value': 'PT0100000022', 'action': '输入'},
             '入会时间_1': {'dom_type': '日期输入框', 'value': '2017-03-15', 'action': '输入'},
             '入会时间_2': {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'},
             '关闭': {'dom_type': '按钮', 'value': '', 'action': '点击'}}

#网页的全部元素
web_element_all = '''可以[点击]名称为[查询]的按钮。可以[输入]名称为[会员代表 会员编号 机构名称]的[输入框]。可以[点击]名称为[机构类型]的[下拉框]，其[下拉框选项]有[全部机构类型 公募基金管理公司 私募基金管理人 证劵公司 私募基金管理人和证劵公司 商业银行 公募基金管理公司子公司 证劵公司资管子公司 期货公司资管子公司 保险公司子公司 独立第三方销售机构 会计师事务所 证劵投资咨询机构 独立服务机构 境外机构 地方自律组织 其他机构]。可以[点击]名称为[会员类型]的[下拉框]，其[下拉框选项]有[全部会员类型 普通会员 联席会员 观察会员 特别会员]。可以[输入]名称为[入会时间_1]的[日期输入框]。可以[输入]名称为[入会时间_2]的[日期输入框]。可以[点击]名称为[关闭]的[按钮]。'''

#网页的部分元素
web_element_part = '''可以[点击]名称为[查询]的按钮。可以[输入]名称为[会员代表 会员编号 机构名称]的[输入框]。可以[输入]名称为[入会时间_1]的[日期输入框]。可以[输入]名称为[入会时间_2]的[日期输入框]。可以[点击]名称为[关闭]的[按钮]。'''

print(encode_test_instruct(web_name, instruct), end='\n\n')
print(encode_test_response(key_value), end='\n\n')
print(decode_response(encode_test_response(key_value)), end='\n\n')
print(encode_train_instruct1(instruct, web_element_all), end='\n\n')
print(encode_train_instruct2(instruct, web_element_part), end='\n\n')
print(encode_train_response(web_name, key_value), end='\n\n')
print(trans_key_values2web_element(key_value))

用户浏览的网页是[中国证券投资基金业协会]，请回答执行什么操作能完成[查询以下内容：会员编号为PT0100000022的机构在六年之内的信息]?

在网页上[点击]名称为[查询]的[按钮];[输入]值[PT0100000022]进名称为[会员编号]的[输入框];[输入]值[2017-03-15]进名称为[入会时间_1]的[日期输入框];[输入]值[2023-03-15]进名称为[入会时间_2]的[日期输入框];[点击]名称为[关闭]的[按钮];

{'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '会员编号': {'dom_type': '输入框', 'value': 'PT0100000022', 'action': '输入'}, '入会时间_1': {'dom_type': '日期输入框', 'value': '2017-03-15', 'action': '输入'}, '入会时间_2': {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}, '关闭': {'dom_type': '按钮', 'value': '', 'action': '点击'}}

用户浏览网页包括的全部元素有:可以[点击]名称为[查询]的按钮。可以[输入]名称为[会员代表 会员编号 机构名称]的[输入框]。可以[点击]名称为[机构类型]的[下拉框]，其[下拉框选项]有[全部机构类型 公募基金管理公司 私募基金管理人 证劵公司 私募基金管理人和证劵公司 商业银行 公募基金管理公司子公司 证劵公司资管子公司 期货公司资管子公司 保险公司子公司 独立第三方销售机构 会计师事务所 证劵投资咨询机构 独立服务机构 境外机构 地方自律组织 其他机构]。可以[点击]名称为[会员类型]的[下拉框]，其[下拉框选项]有[全部会员类型 普通会员 联席会员 观察会员 特别会员]。可以[输入]名称为[入会时间_1]的[日期输入框]。可以[输入]名称为[入会时间_2]的[日期输入框]。可以[点击]名称为[关闭]的[按钮]。请先回答这是什么网页？再回答执行什么操作能完成[查询以下内容：会员编号为PT0100000022的机构在六年之内的信息]？

用户浏览网页包括的部分元素有:可以[点击

In [ ]:
# 构建样本 prompt
'''
1.网页元素（不告诉web) + instruction -> response（回答web+操作) (len<512)
2.使用的网页元素(不告诉web) + instruction -> response（回答web+操作）
3.web名称 + instruction  ->  response(操作，不回答web)
'''

import random

def generate_train_dataset(label_trainset,
                           add_data,
                           web_name,
                           web_element_all,
                           rule_num1=2000,
                           rule_num2=2000,
                           rule_num3=2000,
                           common_num=500,
                           text_max_len=512):

    '''
    rule_num1:  规则1生成样本数目
    rule_num2:  规则2生成样本数目
    rule_num3:  规则3生成样本数目
    common_num:  规则1、规则2、规则3共同样本数目
    默认：  comon_num < rule_num1 + rule_num2 + rule_num3 < len(add_data)
    '''

    random.shuffle(label_trainset)
    random.shuffle(add_data)
    train_rule1, train_rule2, train_rule3 = [], [], []
    for data in label_trainset:
        instruct_add = encode_train_instruct1(data['instruction'], web_element_all)
        if len(instruct_add) < text_max_len and len(train_rule1) < rule_num1:
            train_rule1.append({
                'prompt': instruct_add,
                'response': encode_train_response(web_name, data['key-value']),
                'history': []
            })

        if len(train_rule2) < rule_num2:
            train_rule2.append({
                'prompt': encode_train_instruct2(data['instruction'],
                                                 trans_key_values2web_element(data['key-value'])),
                'response': encode_train_response(web_name, data['key-value']),
                'history': []
            })

        if len(train_rule3) < rule_num3:
            train_rule3.append({
                'prompt': encode_test_instruct(web_name, data['instruction']),
                'response': encode_test_response(data['key-value']),
                'history': []
            })
    print(len(train_rule1), len(train_rule2), len(train_rule3))

    start_index = max(0, common_num - len(label_trainset))
    for data in add_data[:start_index]:
        instruct_add = encode_train_instruct1(data['instruction'], web_element_all)
        if len(instruct_add) < text_max_len and len(train_rule1) < rule_num1:
            train_rule1.append({
                'prompt': instruct_add,
                'response': encode_train_response(web_name, data['key-value']),
                'history': []
            })

        if len(train_rule2) < rule_num2:
            train_rule2.append({
                'prompt': encode_train_instruct2(data['instruction'],
                                                 trans_key_values2web_element(data['key-value'])),
                'response': encode_train_response(web_name, data['key-value']),
                'history': []
            })

        if len(train_rule3) < rule_num3:
            train_rule3.append({
                'prompt': encode_test_instruct(web_name, data['instruction']),
                'response': encode_test_response(data['key-value']),
                'history': []
            })
    print(len(train_rule1), len(train_rule2), len(train_rule3))

    left_index3 = rule_num3 - len(train_rule3)
    for index in add_data[start_index: start_index+left_index3]:
        train_rule3.append({
            'prompt': encode_test_instruct(web_name, data['instruction']),
            'response': encode_test_response(data['key-value']),
            'history': []
        })
    start_index += left_index3
    print(len(train_rule1), len(train_rule2), len(train_rule3))

    left_index2 = rule_num2 - len(train_rule2)
    for index in add_data[start_index: start_index+left_index2]:
        train_rule2.append({
            'prompt': encode_train_instruct2(data['instruction'],
                                             trans_key_values2web_element(data['key-value'])),
            'response': encode_train_response(web_name, data['key-value']),
            'history': []

        })
    start_index += left_index2
    print(len(train_rule1), len(train_rule2), len(train_rule3))

    for index in add_data[start_index:]:
        instruct_add = encode_train_instruct1(data['instruction'], web_element_all)
        if len(instruct_add) < text_max_len:
            train_rule1.append({
                'prompt': instruct_add,
                'response': encode_train_response(web_name, data['key-value']),
                'history': []
            })
        if len(train_rule1) >= rule_num1:
            break
    print(len(train_rule1), len(train_rule2), len(train_rule3))

    train_rule1.extend(train_rule2)
    train_rule1.extend(train_rule3)
    print("all data num:", len(train_rule1))
    random.shuffle(train_rule1)
    return train_rule1

# Load chat-glm fine-tune Model

In [ ]:
import os
os.chdir("")
print("Current Run Path: ", os.path.abspath('.'))
os.system("pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple")

import json
# 首先载入Tokenizer
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

local_model_path = ""
# 加载 Checkpoint
config = AutoConfig.from_pretrained(local_model_path, trust_remote_code=True)
config.pre_seq_len = 128

tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(local_model_path, trust_remote_code=True, config=config).half()
model = model.to("cuda:0")

# 本次微调得到的glm权重
fine_tune_model_path = ""
prefix_state_dict = torch.load(fine_tune_model_path)
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
#print(new_prefix_state_dict)
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict, strict=False)

# 测试是否部署完成
response, history = model.chat(tokenizer, '''用户使用的网站是[qq音乐]，需求是[查一下摇滚风格的歌手列表]。请问在网页上执行什么操作能完成这个需求？''', history=[])
print(response)


from tqdm import tqdm

LABEL_PATH = ""
print("label_path:", LABEL_PATH)
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json', encoding='utf-8'))
submit_prediction = list()

for idx in tqdm(range(len(instruction_testA))):
    page_predictions = []
    page_source = instruction_testA[idx]['page_source']
    for index, instruction in enumerate(instruction_testA[idx]['instruction_detail']):
        instruct = {}
        query = encode_instruct(page_source, instruction['instruction'])
        #print('query:', query)
        response, history = model.chat(tokenizer, query)
        #print('response:', response)
        key_value = ""
        try:
            key_value = decode_response(response)
        except Exception as e:
            print("instruction:[%s] query:[%s] response:[%s]"% \
                  (instruction['instruction'], query, response))

        instruct['instruction'] = instruction['instruction']
        instruct['key-value'] = key_value
        page_predictions.append(instruct)
        #c = input("??")

    submit_prediction.append({
        'page_source': page_source,
        'instruction_detail': page_predictions
    })

print("submit_prediction [0]:", submit_prediction[0])
#submit_prediction = set(submit_prediction)
with open('submission.json', 'w') as up:
    json.dump(submit_prediction, up)

# 干活啊

## 构造训练数据
根据网页和label_trainset的规则，构造更多数据
## 整理网页信息 prompt:
网页[]包括：可以[点击]名称为[]的[按钮]。可以[输入]名称为[]的[输入框]。可以[输入]名称为[]的[日期输入框]。可以[点击]名称为[]的[下拉框],其中名称是[机构类型]的[下拉框]的选项有[]。



# AIWIN--中文网页自动导航挑战赛—数据构造



## QQ 音乐 By mryxj




### 整理网页信息prompt:
网页[qq音乐]包括：可以[点击]名称为[搜索]的[按钮]。可以[点击]名称为[内地 港台 欧美 日本 韩国]的[单选框]。可以[点击]名称为[男 女 组合]的[单选框]。可以[点击]名称为[流行 说唱 国风 摇滚 电子 民谣 轻音乐 爵士 古典 乡村 蓝调 R&B 民族乐]的[单选框]。

In [ ]:
import random

advs = ["", "请", "帮助", "我要"]
verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]
countries = ["", "内地", "港台", "欧美", "日本", "韩国"]
styles = ["", "流行", "说唱", "国风", "摇滚", "电子", "民谣", "轻音乐", "爵士", "古典", "乡村", "蓝调", "R&B", "民族乐"]
sexs = ["", "男", "女", "组合"]

qq_music_instruction_detail = []

qq_web_element = '''
可以[点击]名称为[搜索]的[按钮]。可以[点击]名称为[内地 港台 欧美 日本 韩国]的[单选框]。可以[点击]名称为[男 女 组合]的[单选框]。可以[点击]名称为[流行 说唱 国风 摇滚 电子 民谣 轻音乐 爵士 古典 乡村 蓝调 R&B 民族乐]的[单选框]。
'''

generate_train1 = []
generate_train2 = []
generate_train3 = []

for adv in advs:
    for verb in verbs:
        for country in countries:
            for sex in sexs:
                for style in styles:
                    sample = {}
                    actions = [country, style, sex]
                    actions = list(filter(lambda x:len(x)>0, actions))
                    Country = country + "地区的" if len(country) > 0 else country
                    Style = style + "风格的" if len(style) > 0 else style
                    if sex == "组合":
                        Sex = sex + "形式的"
                    elif sex == "男" or sex == "女":
                        Sex  = sex + "性的"
                    else:
                        Sex = sex
                    select_rand = random.randint(0, 1)
                    if select_rand % 2 == 0:
                        instruct = adv + verb + Country + Style + Sex + "歌手列表"
                    else:
                        instruct = adv + verb + Sex + Country + Style + "歌手列表"

                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["搜索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    for action in actions:
                        key_value[action] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    sample["key-value"] = key_value
                    # print(sample)
                    qq_music_instruction_detail.append(sample)
                    # c = input("!!!")

print("Generate qq music num: %d" % len(qq_music_instruction_detail))
#qq_music_instruction_detail = random.sample(qq_music_instruction_detail, 1000)
#print(len(qq_music_instruction_detail))

Generate qq music num: 6720


In [ ]:
qq_music_instruction_detail[:10]

[{'instruction': '查找歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'}}},
 {'instruction': '查找流行风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
   '流行': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
 {'instruction': '查找说唱风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
   '说唱': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
 {'instruction': '查找国风风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
   '国风': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
 {'instruction': '查找摇滚风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
   '摇滚': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
 {'instruction': '查找电子风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
   '电子': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
 {'instruction': '查找民谣风格的歌手列表',
  'key-value': {'搜索': {'dom_type': '按钮', 'value'

## 中国证劵投资基金业协 by mryxj
### 网页整理prompt:
网页[中国证券投资基金业协会]包括：可以[输入]名称为[会员代表 会员编号 机构名称]的[输入框]。 可以[点击]名称为[机构类型]的[下拉框]，其[下拉框选项]有[全部机构类型 公募基金管理公司 私募基金管理人 证劵公司 私募基金管理人和证劵公司 商业银行 公募基金管理公司子公司 证劵公司资管子公司 期货公司资管子公司 保险公司子公司 独立第三方销售机构 会计师事务所 证劵投资咨询机构 独立服务机构 境外机构 地方自律组织 其他机构]。可以[点击]名称为[会员类型]的[下拉框]，其[下拉框选项]有[全部会员类型 普通会员 联席会员 观察会员 特别会员]。可以[输入]名称为[入会时间_1]的[日期输入框]。可以[输入]名称为[入会时间_2]的[日期输入框]。








In [ ]:
import random

advs = ["", "请", "帮助", "我要", "帮我"]
verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]
vip_or_organs = ["",
                 "会员代表王基业",
                 "会员代表李长城",
                 "会员代表赵红旗",               # 会员代表
                 "会员编号为PT0120000023的机构", # 会员编号
                 "汇添富基金管理股份有限公司",     # 机构名称
                 "工银瑞信基金管理有限公司"]
vip_types = ["全部会员类型",
             "普通会员",
             "联席会员",
             "观察会员",
             "特别会员",
             "普通会员和特别会员",
             "联席会员和观察会员",
             "普通会员、联席会员和观察会员",
             "联席会员、观察会员和特别会员"]   # 会员类型
organs = ["全部机构类型",
          "公募基金管理公司",
          "私募基金管理人",
          "证劵公司",
          "私募基金管理人和证劵公司"
          "商业银行",
          "公募基金管理公司子公司",
          "证劵公司资管子公司",
          "期货公司资管子公司",
          "保险公司子公司",
          "独立第三方销售机构",
          "会计师事务所",
          "证劵投资咨询机构",
          "独立服务机构",
          "境外机构",
          "地方自律组织",
          "其他机构",
          "保险公司和期货银行",
          "会计师事务所和独立服务机构",
          "证劵公司资管子公司和期货公司资管子公司",
          "证券公司、证券投资咨询机构和私募基金管理人"]  # 机构类型


half_year = ["最近半年的", "在最近半年的", "半年之内的", "在半年之内", "半年期间的", "在半年期间的", "半年内的", "在半年内的", "近半年内的", "近半年间的"]
one_year = ["最近一年的", "在最近一年的", "一年之内的", "在一年之内", "一年期间的", "在一年期间的", "一年内的", "在一年内的", "近一年内的", "近一年间的"]
two_year = ["最近两年的", "在最近两年的", "两年之内的", "在两年之内", "两年期间的", "在两年期间的", "两年内的", "在两年内的", "近两年内的", "近两年间的"]
three_year = ["最近三年的", "在最近三年的", "三年之内的", "在三年之内", "三年期间的", "在三年期间的", "三年内的", "在三年内的", "近三年内的", "近三年间的"]
four_year = ["四年之内的", "在四年之内的", "最近四年的", "四年期间的", "最近四年期间的", "四年间的", "近四年内的", "近四年期间的", "最近四年内的", "近四年间的"]
five_year = ["五年之内的", "在五年之内的", "最近五年的", "五年期间的", "最近五年期间的", "五年间的", "近五年内的", "近五年期间的", "最近五年内的", "近五年间的"]
six_year = ["最近六年的", "六年之内的", "六年内的", "在六年之内的", "在六年内的", "六年期间的", "最近六年期间的", "六年间的", "近六年内的", "近六年期间的"]
five_month =  ["最近五个月的", "在五个月内的", "五个月期间的", "五个月内的", "五个月之内的", "在五个月期间的", "近五个月期间的", "近五个月内的", "近五个月间的"]
seven_month = ["最近七个月的", "在七个月内的", "七个月期间的", "七个月内的", "七个月之内的", "在七个月期间的", "近七个月期间的", "近七个月内的", "近七个月间的"]
ten_month = ["最近十个月的", "在十个月内的", "十个月期间的", "十个月内的", "十个月之内的", "在十个月期间的", "近十个月期间的", "近十个月内的", "近十个月间的"]
times = ["",
         "2018年11月8日到2022年5月20日的",
         "2021年2月15日到2023年9月8日期间的"]   # 结束时间统一是 2023.03.15
times.extend(one_year)
times.extend(two_year)
times.extend(six_year)
times.extend(five_month)
times.extend(ten_month)

asset_china_add = []
for adv in advs:
    for verb in verbs:
        for user_input in vip_or_organs:
            for vip_type in vip_types:
                if len(user_input) > 0 and len(vip_type) > 0: continue
                vip_type = vip_type + "的" if len(vip_type) > 0 else vip_type
                vip_type_tmp = vip_type
                vip_dom = []
                if '、' in vip_type_tmp and '和' in vip_type_tmp:
                    item = vip_type_tmp.split("、")
                    vip_dom.append(item[0])
                    vip_dom.extend(item[1].split("和"))
                elif '和' in vip_type_tmp:
                    vip_dom.extend(vip_type_tmp.split("和"))
                else:
                    vip_dom.append(vip_type_tmp)
                for organ in organs:
                    if len(user_input) > 0 and len(organ) > 0: continue
                    organ = organ + "的" if len(organ) > 0 else organ
                    organ_tmp = organ
                    organ_dom = []
                    if '、' in organ_tmp and '和' in organ_tmp:
                        item = organ_tmp.split("、")
                        organ_dom.append(item[0])
                        organ_dom.extend(item[1].split("和"))
                    elif '和' in organ_tmp:
                        organ_dom.extend(organ_tmp.split("和"))
                    else:
                        organ_dom.append(organ_tmp)
                    for time in times:
                        select_rand = random.randint(0, 2)
                        if select_rand % 3 == 0:
                            instruct = adv + verb + user_input + vip_type + organ + time + '信息'
                        elif select_rand % 3 == 1:
                            instruct = adv + verb + user_input + organ + vip_type + time + "信息"
                        else:
                            instruct = adv + verb + time + user_input + organ + vip_type + "信息"

                        sample = {}
                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        if len(user_input) > 0:
                            if "会员代表" in user_input:
                                key_value_name = "会员代表"
                            elif "会员编号" in user_input:
                                key_value_name = "会员编号"
                            elif "公司" in user_input:
                                key_value_name = "机构名称"
                            key_value[key_value_name] =  {'dom_type': '输入框', 'value': user_input, 'action': '输入'}

                        if len(vip_dom) > 0:
                            key_value['会员类型'] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                        for vip in vip_dom:  #
                            key_value[vip] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}

                        if len(organ_dom) > 0:
                            key_value['机构类型'] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                        for organ in organ_dom:
                            key_value[organ] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}

                        if len(time) > 0:
                            if time in one_year:
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2022-03-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in two_year:
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2021-03-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in six_year:
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2017-03-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in five_month:
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ten_month:
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2022-05-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == "2018年11月8日到2022年5月20日的":
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2018-11-08', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2022-05-20', 'action': '输入'}
                            elif time == "2021年2月15日到2023年9月8日期间的":
                                key_value['入会时间_1'] = {'dom_type': '日期输入框', 'value': '2021-02-15', 'action': '输入'}
                                key_value['入会时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-08', 'action': '输入'}
                        key_value['关闭'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        asset_china_add.append(sample)

print("Generate all num: %d" % len(asset_china_add))
asset_china_add = random.sample(asset_china_add, 20000)
print(len(asset_china_add))


asset_china_web_element = '''[点击][查询][按钮]。[输入][会员代表 会员编号 机构名称][输入框]。[点击][机构类型][下拉框]，其[下拉框选项]有[全部机构类型 公募基金管理公司 私募基金管理人 证劵公司 私募基金管理人和证劵公司 商业银行 公募基金管理公司子公司 证劵公司资管子公司 期货公司资管子公司 保险公司子公司 独立第三方销售机构 会计师事务所 证劵投资咨询机构 独立服务机构 境外机构 地方自律组织 其他机构]。[点击][会员类型][下拉框]，其[下拉框选项]有[全部会员类型 普通会员 联席会员 观察会员 特别会员]。[输入][入会时间_1][日期输入框]。[输入][入会时间_2][日期输入框]。[点击][关闭][按钮]。'''

asset_china_add = generate_train_dataset(label_trainset[5]['instruction_detail'],
                                         asset_china_add,
                                         "中国证券投资基金业协会",
                                         asset_china_web_element,
                                         rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500
                                         )

In [ ]:
len(asset_china_instruction_detail)

31500

In [ ]:
label_trainset[15]

NameError: ignored

In [ ]:


#   B榜数据新版_guo


page_source = "中国证券投资基金业协会"
import random
from random import choice
from datetime import datetime
advs = ["", "请"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data = {}
data["机构名称s"] = "新华基金管理股份有限公司 建信基金管理有限责任公司 华泰柏瑞基金管理有限公司 国海富兰克林基金管理有限公司 汇添富基金管理股份有限公司".split()
data["会员编号s"] = "PT0100000022 PT0100000024 PT0100000025".split()
data["会员代表s"] = "王鸿嫔 杨仓兵 黄小薏".split()
data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'半年': ['2022-09-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15'], '四年': ['2019-03-15', '2023-03-15'], '六年': ['2017-03-15', '2023-03-15']}
data["入会时间_1s"] = "2018-02-10 2021-01-01".split()
data["入会时间_2s"] = "2022-08-20 2023-09-01".split()

data["会员类型s"] = ["全部会员类型",
             "普通会员",
             "联席会员",
             "观察会员",
             "特别会员",
             "普通会员和特别会员",
             "联席会员和观察会员",
             "普通会员、联席会员和观察会员",
             "联席会员、观察会员和特别会员"]   # 会员类型
data["机构类型s"] = ["全部机构类型",
          "公募基金管理公司",
          "私募基金管理人",
          "证劵公司",
          "私募基金管理人和证劵公司"
          "商业银行",
          "公募基金管理公司子公司",
          "证劵公司资管子公司",
          "期货公司资管子公司",
          "保险公司子公司",
          "独立第三方销售机构",
          "会计师事务所",
          "证劵投资咨询机构",
          "独立服务机构",
          "境外机构",
          "地方自律组织",
          "其他机构",
          "保险公司和期货银行",
          "会计师事务所和独立服务机构",
          "证劵公司资管子公司和期货公司资管子公司",
          "证券公司、证券投资咨询机构和私募基金管理人"]  # 机构类型

advs = ["", "请", "帮助", "我要", "帮我"]
verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]

asset_china_instruction_detail = []




for i in range(1,1000000):
    adv = choice(advs)
    verb = choice(verbs)

    机构名称 = choice(data["机构名称s"])
    会员编号 = choice(data["会员编号s"])
    会员代表 = choice(data["会员代表s"])



    会员类型_1 = choice(data["会员类型s"])
    会员类型_2 = choice(data["会员类型s"])
    会员类型_3 = choice(data["会员类型s"])

    机构类型_1 = choice(data["机构类型s"])
    机构类型_2 = choice(data["机构类型s"])
    机构类型_3 = choice(data["机构类型s"])

    入会时间_1 = choice(data["入会时间_1s"])
    入会时间_2 = choice(data["入会时间_2s"])

    year = choice(list(data["years"].keys()))
    val = data["years"][year]

    sample = {}
    key_value = {}

    instruct = adv + verb

    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}



    if(random.random()>0.5):
        key_value["入会时间_1"] = {'dom_type': '日期输入框', 'value': val[0], 'action': '输入'}
        key_value["入会时间_2"] = {'dom_type': '日期输入框', 'value': val[1], 'action': '输入'}
        if(random.random()>0.5):
            instruct += "最近" + year + "之内"
        else:
            instruct += "最近" + year + "的"
    if(random.random() > 0.5 and year not in instruct):
        key_value["入会时间_1"] = {'dom_type': '日期输入框', 'value': 入会时间_1, 'action': '输入'}
        key_value["入会时间_2"] = {'dom_type': '日期输入框', 'value': 入会时间_2, 'action': '输入'}
        入会时间_1 = datetime.strptime(入会时间_1, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
        入会时间_2 = datetime.strptime(入会时间_2, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')

        instruct += (入会时间_1 + "到" + 入会时间_2 + "期间的")


    if(random.random()>0.5):
        key_value["机构名称"] = {'dom_type': '输入框', 'value': 机构名称, 'action': '输入'}
        instruct += 机构名称
    if(random.random()>0.5 and 机构名称 not in instruct):
        key_value["会员编号"] = {'dom_type': '输入框', 'value': 会员编号, 'action': '输入'}
        instruct += "会员编号为" + 会员编号
    if(random.random()>0.5 and 机构名称 not in instruct and 会员编号 not in instruct):
        key_value["会员代表"] = {'dom_type': '输入框', 'value': 会员代表, 'action': '输入'}
        instruct += "会员代表" + 会员代表


    if(random.random()>0.5):
        key_value["会员类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[会员类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 会员类型_1
        if(random.random()>0.5 and 会员类型_1 != 会员类型_2):
            key_value[会员类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "和" + 会员类型_2
        if(random.random()>0.5 and 会员类型_1 != 会员类型_3 and 会员类型_2 != 会员类型_3):
            key_value[会员类型_3] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "和" + 会员类型_3
    if(random.random()>0.5):
        key_value["机构类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[机构类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 机构类型_1
        if(random.random()>0.5 and 机构类型_1 != 机构类型_2):
            key_value[机构类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "和" + 机构类型_2
        if(random.random()>0.5 and 机构类型_2 != 机构类型_3 and 机构类型_1 != 机构类型_3):
            key_value[机构类型_3] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "和" + 机构类型_3

    sample["instruction"] = instruct + "的信息"

    key_value["关闭"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    sample["key-value"] = key_value
    asset_china_instruction_detail.append(sample)
asset_china_instruction_detail = random.sample(asset_china_instruction_detail, 10000)
print(len(asset_china_instruction_detail))
print(asset_china_instruction_detail[-30:])

10000
[{'instruction': '帮助查询一下最近半年之内会员代表王鸿嫔联席会员和普通会员、联席会员和观察会员全部机构类型和期货公司资管子公司和公募基金管理公司的信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '入会时间_1': {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}, '入会时间_2': {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}, '会员代表': {'dom_type': '输入框', 'value': '王鸿嫔', 'action': '输入'}, '会员类型': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '联席会员': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '普通会员、联席会员和观察会员': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '机构类型': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '全部机构类型': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '期货公司资管子公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '公募基金管理公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '关闭': {'dom_type': '按钮', 'value': '', 'action': '点击'}}}, {'instruction': '查询一下联席会员和全部会员类型和联席会员和观察会员的信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '会员类型': {'dom_type': '

## 中国人民银行 by MrYXJ
### 整理网页prompt
网页[中国人民银行]包括：可以[输入]名称为[包含以下全部的关键词]的[输入框]。可以[输入]名称为[包含以下完整关键词]的[输入框]。可以[输入]名称为[包含以下任意一个关键词]的[输入框]。可以[输入]名称为[不包含以下关键词]的[输入框]。可以[点击]名称为[时间不限 一天内 一周内 一月内 一年内]的[单选框]。可以[点击]名称为[默认 标题]的单选框。可以[点击]名称为[请选择查询范围]的[下拉框]，其[下拉框选项]有[全站 中国人民银行 重庆营管部 沈阳分行 大连中心支行 上海总部 天津分行 南京分行 党委宣传部 济南分行 武汉分行 广州分行 成都分行 贵阳中心支行 沟通交流 出版物 长春中心支行 调查统计司 拉萨中心支行 杭州中心支行 西宁中心支行 热点专题 银川中心支行 乌鲁木齐中心支行 合肥中心支行 深圳中心支行 郑州中心支行 研究局 货币政策司 青岛中心支行 福州中心支行 宁波中心支行 厦门中心支行 政务公开 兰州中心支行 哈尔滨中心支行 征信管理局 支付结算司 行领导 西安分行 营业管理部 呼和浩特中心支行 长沙中心支行 英文站点 石家庄中心支行 南昌中心支行 海口中心支行 南宁中心支行 昆明中心支行 太原中心支行 反洗钱局 党校 办公厅_党委办公室 参事室 离退休干部局 国库局 人事司 国际司 人事司 国际司 条法司 金融消费权益保护局 会计财务司 金融稳定局 科技司 团委 金融市场司 内审司 工会 货币金银局 集中采购中心 驻中国人民银行纪检监察组 金融信息中心 宏观审慎管理局 金融相关 机关党委 中国人民银行机关服务中心]


In [ ]:
import random
from tqdm import tqdm

advs = ["", "请", "帮助"]
verbs = ["查找", "查找一下", "查询以下内容："]
has_all_keywords = []
has_complete_keywords = []
has_any_keywords = []
has_not_keywords = []

for data in label_trainset[2]['instruction_detail']:
    for key, value in data['key-value'].items():
        if key == "包含以下全部的关键词":
            has_all_keywords.append(value['value'])
        elif key == "包含以下完整关键词":
            has_complete_keywords.append(value['value'])
        elif key == "包含以下任意一个关键词":
            has_any_keywords.append(value['value'])
        elif key == "不包含以下关键词":
            has_not_keywords.append(value['value'])

keywords = []
keywords.extend(has_all_keywords)
keywords.extend(has_complete_keywords)
keywords.extend(has_any_keywords)
keywords.extend(has_not_keywords)
keywords = list(set(keywords))
keywords.append("")
keywords.append("股市")
#keywords.append("股票涨势、股市")
#keywords.append("人民币、汇率、跌破")
print(keywords)


times = "时间不限 一天内 一周内 一月内 一年内".split(" ")
times.append("")
locations = "默认 标题".split(" ")
locations.append("")
bank_range = "全站 中国人民银行 重庆营管部 沈阳分行 大连中心支行 上海总部 天津分行 南京分行 党委宣传部 济南分行 武汉分行 广州分行 成都分行 贵阳中心支行 沟通交流 出版物 长春中心支行 调查统计司 拉萨中心支行 杭州中心支行 西宁中心支行 热点专题 银川中心支行 乌鲁木齐中心支行 合肥中心支行 深圳中心支行 郑州中心支行 研究局 货币政策司 青岛中心支行 福州中心支行 宁波中心支行 厦门中心支行 政务公开 兰州中心支行 哈尔滨中心支行 征信管理局 支付结算司 行领导 西安分行 营业管理部 呼和浩特中心支行 长沙中心支行 英文站点 石家庄中心支行 南昌中心支行 海口中心支行 南宁中心支行 昆明中心支行 太原中心支行 反洗钱局 党校 办公厅_党委办公室 参事室 离退休干部局 国库局 人事司 国际司 人事司 国际司 条法司 金融消费权益保护局 会计财务司 金融稳定局 科技司 团委 金融市场司 内审司 工会 货币金银局 集中采购中心 驻中国人民银行纪检监察组 金融信息中心 宏观审慎管理局 金融相关 机关党委 中国人民银行机关服务中心"
bank_range = bank_range.split(" ")
bank_range = random.sample(bank_range, 5)
bank_range.append("")
print(times, locations, bank_range)

people_bank_instruction_detail = []

for adv in tqdm(advs):
    for verb in verbs:
        for all_keyword in keywords[:4]:
            all_keyword_instr = "包含全部关键词如“%s”，" % all_keyword if len(all_keyword) > 0 else all_keyword
            for complete_keyword in keywords[4:6]:
                if complete_keyword == all_keyword: continue
                complete_keyword_instr = "包含完整关键词“%s”，" % complete_keyword if len(complete_keyword) > 0 else complete_keyword
                for any_keyword in keywords[6:8]:
                    if complete_keyword == any_keyword or any_keyword == all_keyword: continue
                    any_keyword_instr = "包含某个关键词如“%s”，" % any_keyword if len(any_keyword) > 0 else any_keyword
                    for not_keyword in keywords[8:]:
                        if all_keyword == not_keyword or complete_keyword == not_keyword or any_keyword == not_keyword : continue
                        not_keyword_instr = "不包含关键词如“%s”，" % not_keyword if len(not_keyword) > 0 else not_keyword
                        for location in locations:
                            location_instr = "关键词在%s位置" % location if len(location) > 0 else location
                            for bank_ran in bank_range:
                                bank_ran_instr = "在“%s”范围内查询" %  bank_ran if len(bank_ran) > 0 else bank_ran
                                if len(all_keyword_instr) + len(complete_keyword_instr) + len(any_keyword_instr) + len(not_keyword_instr) == 0 : continue
                                for time in times:
                                    time_instr = "%s" % time if len(time) > 0 else time
                                    sample = {}
                                    key_value = {}
                                    key_value['高级搜索'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                                    if len(all_keyword) > 0:
                                        key_value['包含以下全部的关键词'] = {'dom_type': '输入框', 'value': all_keyword, 'action': '输入'}
                                    if len(complete_keyword) > 0:
                                        key_value['包含以下完整关键词'] = {'dom_type': '输入框', 'value': complete_keyword, 'action': '输入'}
                                    if len(any_keyword) > 0:
                                        key_value['包含以下任意一个关键词'] =  {'dom_type': '输入框', 'value': any_keyword, 'action': '输入'}
                                    if len(not_keyword) > 0:
                                        key_value['不包含以下关键词'] =  {'dom_type': '输入框', 'value': not_keyword, 'action': '输入'}
                                    if len(location) > 0:
                                        key_value[location] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                                    if len(bank_ran) > 0:
                                        key_value['请选择查询范围'] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                        key_value[bank_ran] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                                    if len(time) > 0:
                                         key_value[time] = {'dom_type': '单选框', 'value': '', 'action': '点击'}


                                    if len(bank_ran_instr) > 0 and len(location_instr) > 0:
                                        rand_select = random.randint(0, 1)
                                        if rand_select == 0:
                                            instruct = adv + verb + all_keyword_instr + complete_keyword_instr + any_keyword_instr + not_keyword_instr + location_instr + '，' + bank_ran_instr + '的' + time_instr + '的查询结果'
                                        else:
                                            instruct = adv + verb + all_keyword_instr + complete_keyword_instr + any_keyword_instr + not_keyword_instr + bank_ran_instr + '，' + location_instr + '的' + time_instr + '的查询结果'
                                    elif len(bank_ran_instr) > 0:
                                        instruct = adv + verb + all_keyword_instr + complete_keyword_instr + any_keyword_instr + not_keyword_instr + bank_ran_instr + '的'+ time_instr + '的查询结果'
                                    elif len(location_instr) > 0:
                                        instruct = adv + verb + all_keyword_instr + complete_keyword_instr + any_keyword_instr + not_keyword_instr + location_instr + '的'+ time_instr + '的查询结果'
                                    else:
                                        instruct = adv + verb + all_keyword_instr + complete_keyword_instr + any_keyword_instr + not_keyword_instr + time_instr + '的查询结果'
                                    sample['instruction'] = instruct
                                    sample['key-value'] = key_value
                                    #print(sample)
                                    #c = input("???")
                                    people_bank_instruction_detail.append(sample)


print("Generate pepple bank num: %d" % len(people_bank_instruction_detail))
people_bank_instruction_detail = random.sample(people_bank_instruction_detail, 20000)
print(len(people_bank_instruction_detail))

In [ ]:

#   B榜新数据_guo

page_source = "中国人民银行"
import random
from random import choice
from datetime import datetime
advs = ["", "请"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data = {}
data["包含以下全部的关键词s"] = "基金 征信 汇率 货币 人民币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
data["包含以下完整关键词s"] = "基金 征信 汇率 货币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
data["包含以下任意一个关键词s"] = "基金 征信 汇率 货币 人民币 反洗钱 国际贸易 石油价格 股票涨势 金融市场".split()
data["不包含以下关键词s"] = "跌破 失业率 亚洲市场 势头良好 市场干扰 金融风暴".split()
data["请选择查询范围s"] = "上海总部 调查统计司 中国人民银行 全站 党委宣传部 长春中心支行 出版物 天津分行".split()
data["限定要搜索的内容的时间是s"] = "时间不限 一天内 一周内 一月内 一年内".split()
data["查询关键词位于s"] = "默认 标题".split()

people_bank_instruction_detail = []

for i in range(1,1000000):
    adv = choice(advs)
    verb = choice(verbs)

    包含以下全部的关键词 = choice(data["包含以下全部的关键词s"])
    包含以下完整关键词 = choice(data["包含以下完整关键词s"])
    包含以下任意一个关键词 = choice(data["包含以下任意一个关键词s"])
    不包含以下关键词 = choice(data["不包含以下关键词s"])

    请选择查询范围 = choice(data["请选择查询范围s"])
    限定要搜索的内容的时间是 = choice(data["限定要搜索的内容的时间是s"])
    查询关键词位于 = choice(data["查询关键词位于s"])

    sample = {}
    key_value = {}
    key_value['高级搜索'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct = adv + verb
    if (random.random() > 0.5):
        key_value['包含以下全部的关键词'] = {'dom_type': '输入框', 'value': 包含以下全部的关键词, 'action': '输入'}
        instruct += "包含全部关键词如" + "”" + 包含以下全部的关键词 + "”,"
    if (random.random() > 0.5):
        key_value['包含以下完整关键词'] = {'dom_type': '输入框', 'value': 包含以下完整关键词, 'action': '输入'}
        instruct += "包含完整关键词" + "”" + 包含以下完整关键词 + "”,"
    if (random.random() > 0.5):
        key_value['包含以下任意一个关键词'] =  {'dom_type': '输入框', 'value': 包含以下任意一个关键词, 'action': '输入'}
        instruct += "包含某个关键词" + "”" + 包含以下任意一个关键词 + "”,"
    if (random.random() > 0.5):
        key_value['不包含以下关键词'] =  {'dom_type': '输入框', 'value': 不包含以下关键词, 'action': '输入'}
        instruct += "不包含关键词如" + "”" + 不包含以下关键词 + "”,"
    if (random.random() > 0.5):
        key_value[查询关键词位于] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += "关键词在" + 查询关键词位于 + "位置的"
    if (random.random() > 0.5):
        key_value['请选择查询范围'] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[请选择查询范围] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "”" + 请选择查询范围 + "“范围内查询的"
    if (random.random() > 0.5):
        key_value[限定要搜索的内容的时间是] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += 限定要搜索的内容的时间是 + "的"
    instruct += "查询结果"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    people_bank_instruction_detail.append(sample)
people_bank_instruction_detail = random.sample(people_bank_instruction_detail, 10000)
print(len(people_bank_instruction_detail))
print(people_bank_instruction_detail[-30:])

10000
[{'instruction': '搜索包含全部关键词如”基金”,不包含关键词如”势头良好”,查询结果', 'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '包含以下全部的关键词': {'dom_type': '输入框', 'value': '基金', 'action': '输入'}, '不包含以下关键词': {'dom_type': '输入框', 'value': '势头良好', 'action': '输入'}}}, {'instruction': '请检索一下包含全部关键词如”金融市场”,”中国人民银行“范围内查询的一天内的查询结果', 'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '包含以下全部的关键词': {'dom_type': '输入框', 'value': '金融市场', 'action': '输入'}, '请选择查询范围': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '中国人民银行': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '一天内': {'dom_type': '单选框', 'value': '', 'action': '点击'}}}, {'instruction': '请查一查”长春中心支行“范围内查询的时间不限的查询结果', 'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '请选择查询范围': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '长春中心支行': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '时间不限': {'dom_type': '单选框', 'value': '', 'action': '点击'}}}, {'instruction': '请检索包含全部关键词如”货币”,包含完整关键词”金融市场”,不包含关

## 猫眼电影 by MrYXJ
### 整理网页prompt
网页[猫眼电影]包括：可以[点击]名称为[全部 爱情 喜剧 动画 剧情 恐怖 惊悚 科幻 动作 悬疑 犯罪 冒险 战争 奇幻 运动 家庭 古装 武侠 西部 历史 传记 歌舞 黑色电影 短片 纪录片 戏曲 音乐 灾难 青春 儿童 其他]的[单选框]。可以[点击]名称为[全部 大陆 美国 韩国 日本 中国香港 中国台湾 泰国 印度 法国 英国 俄罗斯 意大利 西班牙 德国 波兰 澳大利亚 伊朗 其他]的[单选框]。可以[点击]名称为[全部 2029 2028 2027 2026 2025 2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2000-2010 90年代 80年代 70年代 更早]的[单选框]。可以[点击]名称为[按热门排序 按时间排序 按评价排序]的[单选框]。


In [ ]:
page_source = "猫眼电影"
import random
from random import choice
from datetime import datetime
advs = ["", "请"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data = {}
data["类型s"] = "动作 其他 科幻 青春 戏曲 短片 恐怖 纪录片 家庭 黑色电影 战争 音乐 武侠 惊悚 动画 悬疑 灾难 西部 儿童".split()
data["区域s"] = "日本 法国 西班牙 伊朗 其他 俄罗斯 澳大利亚 德国 印度 美国 英国 大陆 波兰 泰国".split()
data["年代s"] = "2013 2012 2017 2020 70年代 2011 2014 2018 2000-2010 2015 更早 2019 2023 2021".split()
data["排序方式s"] = "按热门排序 按评价排序 按时间排序".split()


maoyan_instruction_detail = []

for i in range(1,1000000):
    adv = choice(advs)
    verb = choice(verbs)

    类型 = choice(data["类型s"])
    区域 = choice(data["区域s"])
    年代 = choice(data["年代s"])
    排序方式 = choice(data["排序方式s"])

    sample = {}
    key_value = {}
    key_value['检索'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct = adv + verb
    if (random.random() > 0.5):
        key_value['类型'] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += 类型 + "类的"
    if (random.random() > 0.5):
        key_value['排序方式'] =  {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += "按" + 排序方式 + "的"
    if (random.random() > 0.5):
        key_value['区域'] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += 区域 + "地区的"
    if (random.random() > 0.5):
        key_value['年代'] =  {'dom_type': '单选框', 'value': '', 'action': '点击'}
        if("年" not in instruct):
            instruct += 年代 + "年的"
        else:
            instruct += 年代
    instruct += "的电影"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    maoyan_instruction_detail.append(sample)
maoyan_instruction_detail = random.sample(maoyan_instruction_detail, 10000)
print(len(maoyan_instruction_detail))
print(maoyan_instruction_detail[-30:])

10000
[{'instruction': '请查找法国地区的2020年的的电影', 'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '区域': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '年代': {'dom_type': '单选框', 'value': '', 'action': '点击'}}}, {'instruction': '请搜索动作类的按按时间排序的2020年的的电影', 'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '类型': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '排序方式': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '年代': {'dom_type': '单选框', 'value': '', 'action': '点击'}}}, {'instruction': '请检索一下按按热门排序的德国地区的的电影', 'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '排序方式': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '区域': {'dom_type': '单选框', 'value': '', 'action': '点击'}}}, {'instruction': '请查一查动作类的按按时间排序的美国地区的的电影', 'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '类型': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '排序方式': {'dom_type': '单选框', 'value': '', 'action': '点击'}, '区域': {'dom_type': '单选框', 'value': '',

## 巨潮查询 code by 鼠鼠

In [ ]:
#巨潮查询
advs = ["", "请", "帮助", "我要"]
verbs = ["查找", "查询", "查找一下", "查询一下", "查一查", "查询以下内容：", ""]
板块s = "深泸京 基金 债券 港股 三板".split()

代码s = "000001 000002 000003 000004 000005 000006 000007 000008 000009 000010 000011".split() #+ [""]
简称s = "平安银行 万科A ST国华 ST星源 深振业A 全新好 神州高铁 中国宝安 美丽生态 深物业A".split() #+ [""]
拼音s = "pinganyinhang wankeA STguohua STxingyuan shenzhenyeA quanxinhao shenzhougaotie zhongguobaoan meilishengtai shenwuyeA".split() #+ [""]

深泸京分类s = "年报 半年报 一季报 三季报 业绩预告 权益分派 董事会 监事会 股东大会 日常经营 公司治理 中介报告 \
首发 增发 股权激励 配股 解禁 公司债 可转债 其他融资 股权变动 补充更正 澄清致歉 风险提示 特别处理和退市 退市整理期 \
".split() #+ [""]
基金分类s = "招募设立 说明书更新 年报 中报 季报 净值 投资组合 申购赎回 基金费率 销售渠道 分红 高管及基金经理 持有人大会  \
基本信息变更 其他 \
".split() #+ [""]
债券分类s = "债券发行上市 债券定期公告 债券付息公告 债券到期兑付停 债券其他公告".split() #+ [""]
三板分类s = "临时公告 定期公告 中介机构公告 持续信息披露 首次信息披露".split() #+ [""]
标题关键词s = "\
平安银行 万科A ST国华 ST星源 深振业A 全新好 神州高铁 中国宝安 美丽生态 深物业A \
".split() + [""]
日期s = ["一年内", ""]
now = '2023-03-15'

巨潮查询_instruction_detail = []
for adv in advs:
    for verb in verbs:
        for 板块 in 板块s:
            for i,代码简称拼音 in enumerate(代码s+简称s+拼音s+[""]):
                if 代码简称拼音 in 代码s:
                    signal = '代码'
                    代码简称拼音_ = '代码为' + 代码简称拼音 + '的公司的'
                elif 代码简称拼音 in 简称s:
                    signal = '简称和关键词'
                    代码简称拼音_ = 代码简称拼音 + '的'
                elif 代码简称拼音 in 拼音s:
                    signal = '拼音'
                    代码简称拼音_ = '拼音为' + 代码简称拼音 + '的公司的'
                else:
                    signal = ''

                for 日期 in 日期s:
                    if (日期 != ''):
                        日期 = 日期 + '的'
                    if 板块 == "深泸京":
                        分类s = 深泸京分类s
                    elif 板块 == "基金":
                        分类s = 基金分类s
                    elif 板块 == "债券":
                        分类s = 债券分类s
                    elif 板块 == "港股":
                        分类s = ["公告"]
                    elif 板块 == "三板":
                        分类s = 三板分类s
                    for 分类 in 分类s:
                        samples = {}
                        samples['instructions'] = []
                        samples['key-values'] = []
                        if (signal == '代码'):
                            instruction = adv + verb + 日期 + 代码简称拼音_ + 板块 + 分类
                            samples['instructions'] += [instruction]
                            if (日期 != ''):
                                instruction = adv + verb + 代码简称拼音_ + 日期 + 板块 + 分类
                                samples['instructions'] += [instruction]
                        elif (signal == '简称和关键词'):
                            instruction = adv + verb + 日期 + 代码简称拼音_ + 板块 + 分类
                            samples['instructions'] += [instruction]
                            if (日期 != ''):
                                instruction = adv + verb + 代码简称拼音_ + 日期 + 板块 + 分类
                                samples['instructions'] += [instruction]
                        elif (signal == '拼音'):
                            instruction = adv + verb + 日期 + 代码简称拼音_ + 板块 + 分类
                            samples['instructions'] += [instruction]
                            if (日期 != ''):
                                instruction = adv + verb + 代码简称拼音_ + 日期 + 板块 + 分类
                                samples['instructions'] += [instruction]
                        else:
                            instruction = adv + verb + 日期 + 板块 + 分类
                            samples['instructions'] += [instruction]
                            if (日期 != ''):
                                instruction = adv + verb + 板块 + 日期 + 分类
                                samples['instructions'] += [instruction]

                        key_value = {}
                        key_value['查询'] = {"dom_type": "按钮", "value": "", "action": "点击"}
                        key_value[板块] = {"dom_type": "按钮", "value": "", "action": "点击"}
                        if (代码简称拼音 != ''):
                            key_value['代码/简称/拼音'] = {"dom_type": "输入框", "value": 代码简称拼音, "action": "输入"}
                        if (板块 != '港股' and 分类 != ''):
                            key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                            key_value[分类] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                            key_value['确认'] = {"dom_type": "按钮", "value": "", "action": "点击"}
                        if (日期 != ''):
                            key_value['开始日期'] = {"dom_type": "日期输入框", "value": "2022-03-15", "action": "输入"} # ???
                            key_value['结束日期'] = {"dom_type": "日期输入框", "value": now, "action": "输入"} # ???
                        samples['key-values'] += [key_value]

                        if (signal == '简称和关键词'): # 添加关键词搜索的key-value序列
                            key_value = {}
                            key_value['查询'] = {"dom_type": "按钮", "value": "", "action": "点击"}
                            key_value[板块] = {"dom_type": "按钮", "value": "", "action": "点击"}
                            key_value["标题关键字"] = {"dom_type": "输入框", "value": "深振业A", "action": "输入"}
                            if (分类 != ''):
                                key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
                                key_value[分类] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                                key_value['确认'] = {"dom_type": "按钮", "value": "", "action": "点击"}
                            if (日期 != ''):
                                key_value['开始日期'] = {"dom_type": "日期输入框", "value": "2022-03-15", "action": "输入"} # ???
                                key_value['结束日期'] = {"dom_type": "日期输入框", "value": now, "action": "输入"} # ???
                            samples['key-values'] += [key_value]

                        for instruction in samples['instructions']:
                            for key_value in samples['key-values']:
                                sample = {}
                                sample['instruction'] = instruction
                                sample['key-value'] = key_value
                                巨潮查询_instruction_detail += [sample]

# #print(len(巨潮查询_instruction_detail))
# #print(巨潮查询_instruction_detail[:10])
# import json
# obj = open('巨潮查询_instruction_detail.json', 'w')
# json.dump(巨潮查询_instruction_detail, obj)

In [ ]:

#  B榜新数据_guo

page_source = "巨潮资讯"
import random
from random import choice
from datetime import datetime
advs = ["", "请"]
verbs = ["查找","查询","搜索","检索","查一查","查一下","检索一下","查询一下","查查","检索一下","查找出"]
data = {}
data["板块s"] = "深户京 基金 债券 港股 三板".split()
data["开始日期s"] = "2021-01-01 2022-02-10 ".split()
data["结束日期s"] = "2023-01-01 2023-03-20".split()
data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'一年': ['2022-03-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15']}

data["代码/简称/拼音s"] = "000009 000011 000002 000007 000010 000006 000008 000005".split()
data["标题关键字s"] = "ST国华 深物业A 神州高铁 平安银行 ST星源 深振业A 美丽生态 中国宝安 万科A 全新好".split()
data["深户京s"] = "澄清致歉 中介报告 配股 其他融资 股东大会 风险提示 股权变动 特别处理和退市 三季报 退市整理期 年报 董事会 权益分派 增发 一季报 首发 公司债".split()
data["基金s"] = "基金费率 申购赎回 中报 分红 其他 投资组合 季报 基本信息变更 年报 持有人大会 说明书更新 高管及基金经理 招募设立 销售渠道".split() #+ [""]
data["债券s"] = "债券付息公告 债券发行上市 债券定期公告 债券到期兑付停".split() #+ [""]
data["三板s"] = "中介机构公告 首次信息披露 定期公告 持续信息披露 临时公告".split() #+ [""]


巨潮查询_instruction_detail = []

for i in range(1,1000000):
    adv = choice(advs)
    verb = choice(verbs)

    板块 = choice(data["板块s"])

    代码 = choice(data["代码/简称/拼音s"])
    标题关键字 = choice(data["标题关键字s"])


    深户京_1 = choice(data["深户京s"])
    基金_1 = choice(data["基金s"])
    债券_1 = choice(data["债券s"])
    三板_1 = choice(data["三板s"])

    深户京_2 = choice(data["深户京s"])
    基金_2 = choice(data["基金s"])
    债券_2 = choice(data["债券s"])
    三板_2 = choice(data["三板s"])

    year = choice(list(data["years"].keys()))
    val = data["years"][year]

    开始日期 = choice(data["开始日期s"])
    结束日期 = choice(data["结束日期s"])


    sample = {}
    key_value = {}
    key_value['查询'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct = adv + verb
    key_value[板块] = {'dom_type': '按钮', 'value': '', 'action': '点击'}

    if(random.random()>0.5):
        key_value["标题关键字"] = {"dom_type": "输入框", "value": 标题关键字, "action": "输入"}
        instruct += 标题关键字
    if(random.random()>0.5 and 标题关键字 not in instruct):
        key_value['代码/简称/拼音'] = {"dom_type": "输入框", "value": 代码, "action": "输入"}
        instruct += "代码为" + 代码 + "公司的"
    if(random.random()>0.5):
        key_value['开始日期'] = {"dom_type": "日期输入框", "value": 开始日期, "action": "输入"} # ???
        key_value['结束日期'] = {"dom_type": "日期输入框", "value": 结束日期, "action": "输入"} # ???
        开始日期 = datetime.strptime(开始日期, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
        结束日期 = datetime.strptime(结束日期, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
        instruct += 开始日期 + "到" + 结束日期 + "期间的"
    if(random.random()>0.5 and "开始日期" not in instruct):
        key_value['开始日期'] = {"dom_type": "日期输入框", "value": val[0], "action": "输入"} # ???
        key_value['结束日期'] = {"dom_type": "日期输入框", "value": val[1], "action": "输入"} # ???
        instruct +=  "最近" + year

    if(板块 == "深户京"):
        if(random.random()>0.5):
            key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
            key_value[深户京_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
            instruct += "深户京的" +  深户京_1
            if(random.random()>0.5):
                key_value[深户京_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "和" +  深户京_2
        else:
            instruct += "深户京的公告"

    elif(板块 == "基金"):
        if(random.random()>0.5):
            key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
            key_value[基金_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
            instruct += "基金的" + 基金_1
            if(random.random()>0.5):
                key_value[基金_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "和" +  基金_2
        else:
            instruct += "基金的公告"
    elif(板块 == "债券"):
        if(random.random()>0.5):
            key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
            key_value[债券_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
            instruct += 债券_1
            if(random.random()>0.5):
                key_value[债券_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "和" +  债券_2
        else:
            instruct += "债券的公告"
    elif(板块 == "三板"):
        if(random.random()>0.5):
            key_value["分类"] = {"dom_type": "下拉框", "value": "", "action": "点击"}
            key_value[三板_1] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
            instruct += "三板" +  三板_1
            if(random.random()>0.5):
                key_value[三板_2] = {"dom_type": "下拉框选项", "value": "", "action": "点击"}
                instruct += "和" +  三板_2
        else:
            instruct += "三板的公告"
    elif(板块 == "港股"):
        instruct += "公告"
    key_value['确认'] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    巨潮查询_instruction_detail.append(sample)
maoyan_instruction_detail = random.sample(巨潮查询_instruction_detail, 10000)
print(len(巨潮查询_instruction_detail))
print(巨潮查询_instruction_detail[-30:])

999999
[{'instruction': '检索一下平安银行深户京的董事会和配股', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '深户京': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '标题关键字': {'dom_type': '输入框', 'value': '平安银行', 'action': '输入'}, '分类': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '董事会': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '配股': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '确认': {'dom_type': '按钮', 'value': '', 'action': '点击'}}}, {'instruction': '请查一下ST国华深户京的公告', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '深户京': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '标题关键字': {'dom_type': '输入框', 'value': 'ST国华', 'action': '输入'}, '确认': {'dom_type': '按钮', 'value': '', 'action': '点击'}}}, {'instruction': '检索一下全新好2022年2月10日到2023年1月1日期间的最近五个月债券发行上市和债券付息公告', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '债券': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '标题关键字': {'dom_type': '输入框', 'value': '全新好', 'action': '输入'}, '开始

## 中国裁判文书网 code by guo




## 整理网页信息 prompt:
网页[中国裁判文书网]包括：可以[输入]名称为[全文检索]的[输入框]。可以[点击]名称为[全文]的[下拉框]，其[下拉框选项]有[首部 当事人段 诉讼记录 事实 理由 判决结果 尾部 其他]。可以[点击]名称为[案由]的[下拉框]，其[下拉框选项]有[侵占罪 决水罪 分子罪 受贿罪 叛逃罪 失火罪 帮助罪 投毒罪 投降罪 抢劫罪 抢夺罪 放火罪 活动罪 渎职罪 爆炸罪 盗窃罪 行为罪 行贿罪 诈骗罪 贪污罪 资敌罪 走私罪 间谍罪 妨害国(边)境管理罪 人事争议 典当纠纷 反革命罪 合同纠纷 垄断纠纷 帮助犯罪 执行案由 物权纠纷 经济犯罪 继承纠纷 行政案由 交通肇事罪 人格权纠纷 介绍贿赂罪 传播性病罪 侵犯财产罪 军人叛逃罪 分裂国家罪 单位受贿罪 单位行贿罪 危险驾驶罪 名反革命罪 商检失职罪 妨害司法罪 妨害清算罪 嫖宿幼女罪 强迫卖淫罪 徇私枉法罪 战时自伤罪 所有权纠纷 投敌叛变罪 挪用公款罪 挪用资金罪 放纵走私罪 敲诈勒索罪 枉法仲裁罪 枉法裁判罪 滥用职权罪 玩忽职守罪 私放俘虏罪 组织卖淫罪 职务侵占罪 聚众哄抢罪 背叛国家罪 虐待俘虏罪 虐待部属罪 虚假破产罪 贪污贿赂罪 走私假币罪 走私废物罪 走私文物罪 过失决水罪 过失投毒罪 过失爆炸罪 追偿权纠纷 逃离部队罪 金融诈骗罪 银行卡纠纷 放行偷越国(边)境人员罪 办理偷越国(边)境人员出入境证件罪 不当得利纠纷 买卖合同纠纷 仓储合同纠纷 侵权责任纠纷 保管合同纠纷 保证合同纠纷 借款合同纠纷 借用合同纠纷 其他刑事犯罪 其他民事纠纷 劫持航空器罪 劳务合同纠纷 劳动合同纠纷 包庇毒品犯罪 占有保护纠纷 合伙协议纠纷 国家赔偿案由 委托合同纠纷 婚姻家庭纠纷 定金合同纠纷 对单位行贿罪 居间合同纠纷 展览合同纠纷 广告合同纠纷 悬赏广告纠纷 承揽合同纠纷 抵押合同纠纷 担保物权纠纷 拍卖合同纠纷 无因管理纠纷 服务合同纠纷 海事海商纠纷 演出合同纠纷 物权保护纠纷 犯逃避处罚罪 用益物权纠纷 社会保险纠纷 福利待遇纠纷 租赁合同纠纷 行纪合同纠纷 补偿贸易纠纷 质押合同纠纷 赠与合同纠纷 走私核材料罪 运输合同纠纷 不动产登记纠纷 不正当竞争纠纷 丢失枪支不报罪 传播淫秽物品罪 供用气合同纠纷 供用水合同纠纷 供用电合同纠纷 侵犯知识产权罪 军人违反职责罪 分子逃避处罚罪 协助组织卖淫罪 危害公共卫生罪 危害公共安全罪 危害国家安全罪 危害国防利益罪 危害税收征管罪 危险物品肇事罪 商检徇私舞弊罪 妨害文物管理罪 容留他人吸毒罪 帮助恐怖活动罪 引诱幼女卖淫罪 强迫他人吸毒罪 彩票、奖券纠纷 战时临阵脱逃罪 战时违抗命令罪 战时造谣惑众罪 扰乱公共秩序罪 扰乱市场秩序罪 投放危险物质罪 挪用特定款物罪 操纵上市公司罪 故意毁坏财物罪 武器装备肇事罪 消防责任事故罪 煽动分裂国家罪 环境监管失职罪 破坏交通工具罪 破坏交通设施罪 破坏生产经营罪 破坏电力设备罪 私分国有资产罪 私分罚没财物罪 私放在押人员罪 组织淫秽表演罪 虚报注册资本罪 资助恐怖活动罪 走私制毒物品罪 走私固体废物罪 走私淫秽物品罪 走私贵重金属罪 进出口押汇纠纷 违令作战消极罪 遗失武器装备罪 遗弃伤病军人罪 遗弃武器装备罪 重大责任事故罪 重大飞行事故罪 阻碍军事行动罪 隐瞒境外存款罪 非法持有毒品罪 颠覆国家政权罪 食品监管渎职罪 临时用地合同纠纷 为亲友非法牟利罪 人事争议人事争议 传染病防治失职罪 供用热力合同纠纷 债务转移合同纠纷 债权人代位权纠纷 债权人撤销权纠纷 债权转让合同纠纷 储蓄存款合同纠纷 养老保险待遇纠纷 农业承包合同纠纷 利用影响力受贿罪 动植物检疫失职罪 劫持船只、汽车罪 医疗保险待遇纠纷 失业保险待遇纠纷 委托理财合同纠纷 工伤保险待遇纠纷 建设工程合同纠纷 房屋买卖合同纠纷 拒传、假传军令罪 接送不合格兵员罪 林业承包合同纠纷 武装叛乱、暴乱罪 渔业承包合同纠纷 牧业承包合同纠纷 生产、销售假药罪 生产、销售劣药罪 生育保险待遇纠纷 知识产权合同纠纷 确认合同效力纠纷 确认合同无效纠纷 确认合同有效纠纷 缔约过失责任纠纷 融资租赁合同纠纷 走私武器、弹药罪 隐瞒、谎报军情罪 冒充军人招摇撞骗罪 准备实施恐怖活动罪 妨害社会管理秩序罪 婚姻家庭、继承纠纷 工程重大安全事故罪 巨额财产来源不明罪 强令违章冒险作业罪 战时军人造谣惑众罪 战时拒绝军事征用罪 战时造谣扰乱军心罪 拒不支付劳动报酬罪 拒不救援友邻部队罪 指使操纵上市公司罪 指使部属违反职责罪 探矿权转让合同纠纷 提供虚假财会报告罪 故意泄露军事秘密罪 故意泄露国家秘密罪 暴力危及飞行安全罪 民事案由人格权纠纷 煽动军人逃离部队罪 煽动颠覆国家政权罪 知识产权与竞争纠纷 破坏易燃易爆设备罪 破坏环境资源保护罪 破坏金融管理秩序罪 聚众冲击军事禁区罪 过失投放危险物质罪 过失损坏交通工具罪 过失损坏交通设施罪 过失损坏电力设备罪 过失泄露军事秘密罪 过失泄露国家秘密罪 采矿权转让合同纠纷 重大劳动安全事故罪 铁路运营安全事故罪 阻碍军人执行职务罪 阻碍执行军事职务罪 雇用逃离部队军人罪 非法买卖制毒物品罪 非法经营同类营业罪 非法获取军事秘密罪 不报、谎报安全事故罪 公司、企业人员受贿罪 农村土地承包合同纠纷 动植物检疫徇私舞弊罪 劳动争议劳动合同纠纷 对有影响力的人行贿罪 战时拒绝、逃避服役罪 执行判决、裁定失职罪 招标投标买卖合同纠纷 擅离、玩忽军事职守罪 放纵制售伪劣商品犯罪 欺诈发行股票、债券罪 民事、行政枉法裁判罪 生产、销售伪劣产品罪 生产、销售伪劣商品罪 签订、履行合同失职罪 虚假出资、抽逃出资罪 资助危害国家安全犯罪 走私普通货物、物品罪 违规制造、销售枪支罪 适用特殊程序案件案由 非国家工作人员受贿罪 非法出租、出借枪支罪 非法种植毒品原植物罪 适用特殊程序案件案由(2008-2011)——历史案由其他民事纠纷 失职致使在押人员脱逃罪 对公司、企业人员行贿罪 对非国家工作人员行贿罪 建设用地使用权合同纠纷 引诱、容留、介绍卖淫罪 战时拒不救治伤病军人罪 战时故意提供虚假敌情罪 战时窝藏逃离部队军人罪 房地产开发经营合同纠纷 教育设施重大安全事故罪 知识产权权属、侵权纠纷 离退休人员返聘合同纠纷 种植、养殖回收合同纠纷 组织播放淫秽音像制品罪 背信损害上市公司利益罪 过失损坏易燃易爆设备罪 违法提供出口退税凭证罪 非法买卖、运输核材料罪 以危险方法危害公共安全罪 债权债务概括转移合同纠纷 徇私舞弊不征、少征税款罪 徇私舞弊不移交刑事案件罪 徇私舞弊造成破产、亏损罪 战时拒绝军事征收、征用罪 房屋拆迁安置补偿合同纠纷 执行判决、裁定滥用职权罪 抢劫枪支、弹药、爆炸物罪 滥用管理公司、证券职权罪 签订、履行合同失职被骗罪 聚众扰乱军事管理区秩序罪 违法发放林木采伐许可证罪 隐匿、故意销毁会计资料罪 非法出卖、转让武器装备罪 非法批准征用、占用土地罪 非法生产、买卖军用标志罪 九七年十月以前刑事案由或罪 利用极端主义破坏法律实施罪 制作、贩卖、传播淫秽物品罪 引诱、教唆、欺骗他人吸毒罪 招收公务员、学生徇私舞弊罪 擅自出卖、转让军队房地产罪 擅自改变武器装备编配用途罪 生产、销售有毒、有害食品罪 破坏社会主义市场经济秩序罪 组织、领导、参加恐怖组织罪 违规披露、不披露重要信息罪 非法持有、私藏枪支、弹药罪 为他人提供书号出版淫秽书刊罪 人事争议人事争议海事海商纠纷 侵犯公民人身权利、民主权利罪 合同、无因管理、不当得利纠纷 大型群众性活动重大安全事故罪 失职造成珍贵文物损毁、流失罪 妨害对公司、企业的管理秩序罪 战时拒绝、故意延误军事订货罪 战时残害居民、掠夺居民财物罪 窝藏、转移、隐瞒毒品、毒赃罪 走私、贩卖、运输、制造毒品罪 走私珍稀植物、珍稀植物制品罪 走私珍贵动物、珍贵动物制品罪 过失以危险方法危害公共安全罪 非法低价出让国有土地使用权罪 非法提供麻醉药品、精神药品罪 不解救被拐卖、绑架妇女、儿童罪 战时拒绝、逃避征召、军事训练罪 盗窃、抢夺枪支、弹药、爆炸物罪 盗窃、抢夺武器装备、军用物资罪 非法批准征收、征用、占用土地罪 中外合作勘探开发自然资源合同纠纷 徇私舞弊低价折股、出售国有资产罪 生产、销售不符合卫生标准的食品罪 生产、销售不符合安全标准的产品罪 生产、销售不符合安全标准的食品罪 生产、销售不符合标准的医用器材罪 破坏广播电视设施、公用电信设施罪 走私国家禁止进出口的货物、物品罪 阻碍解救被拐卖、绑架妇女、儿童罪 非法生产、买卖武装部队制式服装罪 国有公司、企业、事业单位人员失职罪 徇私舞弊减刑、假释、暂予监外执行罪 抢劫枪支、弹药、爆炸物、危险物质罪 故意提供不合格武器装备、军事设施罪 生产、销售不符合卫生标准的化妆品罪 破坏武器装备、军事设施、军事通信罪 组织、强迫、引诱、容留、介绍卖淫罪 过失提供不合格武器装备、军事设施罪 国家机关工作人员签订、履行合同失职罪 盗窃、抢夺武装部队公文、证件、印章罪 过失损坏广播电视设施、公用电信设施罪 非法制造、买卖、运输、储存危险物质罪 非法持有宣扬恐怖主义、极端主义物品罪 劳动争议、人事争议劳动争议劳动合同纠纷 国有公司、企业、事业单位人员滥用职权罪 对外国公职人员、国际公共组织官员行贿罪 徇私舞弊发售发票、抵扣税款、出口退税罪 生产、销售伪劣农药、兽药、化肥、种子罪 过失损坏武器装备、军事设施、军事通信罪 过失破坏武器装备、军事设施、军事通信罪 与公司、证券、保险、票据等有关的民事纠纷 国家机关工作人员签订、履行合同失职被骗罪 盗窃、抢夺枪支、弹药、爆炸物、危险物质罪 为境外窃取、刺探、收买、非法提供军事秘密罪 伪造、变造、买卖武装部队公文、证件、印章罪 制作、复制、出版、贩卖、传播淫秽物品牟利罪 宣扬恐怖主义、极端主义、煽动实施恐怖活动罪 强制穿戴宣扬恐怖主义、极端主义服饰、标志罪 非法生产、买卖、运输制毒物品、走私制毒物品罪 为境外窃取、刺探、收买、非法提供国家秘密、情报罪 隐匿、故意销毁会计凭证、会计账簿、财务会计报告罪 非法买卖、运输、携带、持有毒品原植物种子、幼苗罪 非法制造、买卖、运输、邮寄、储存枪支、弹药、爆炸物罪 非法携带枪支、弹药、管制刀具、危险物品危及公共安全罪 伪造、盗窃、买卖、非法提供、非法使用武装部队专用标志罪]。可以[输入]名称为[案件名称]的[输入框]。可以[输入]名称为[案号]的[输入框]。可以[输入]名称为[法院名称]的[输入框]。可以[点击]名称为[法院层级]的[下拉框]，其[下拉框选项]有[最高法院 高级法院 中级法院 基层法院]。可以[点击]名称为[案件类型]的[下拉框]，其[下拉框选项]有[管辖案件 刑事案件 民事案件 行政案件 国家赔偿与司法救助案件 区际司法协助案件 国际司法协助案件 司法制裁案件 非诉保全审查案件 执行案件 强制清算与破产案件 其他案件]。可以[点击]名称为[审判程序]的[下拉框]，其[下拉框选项]有[刑事提级管辖 刑事指定管辖 民事提级管辖 民事指定管辖 民事移交管辖审批 民事管辖协商 民事管辖上诉 民事管辖监督 行政提级管辖 行政指定管辖 行政管辖上诉 行政赔偿提级管辖 行政赔偿指定管辖 行政赔偿管辖协商 行政赔偿管辖上诉]。可以[点击]名称为[文书类型]的[下拉框]，其[下拉框选项]有[判决书 裁定书 决定书 通知书 其他]。可以[输入]名称为[裁判日期_1]的[日期输入框]。可以[输入]名称为[裁判日期_2]的[日期输入框]。可以[点击]名称为[案例等级]的[下拉框]，其[下拉框选项]有[指导性案例 优秀文书]。可以[点击]名称为[公开类型]的[下拉框]，其[下拉框选项]有[文书公开 信息公开]。可以[输入]名称为[审判人员]的[输入框]。可以[输入]名称为[当事人]的[输入框]。可以[输入]名称为[律所]的[输入框]。可以[输入]名称为[律师]的[输入框]。可以[输入]名称为[法律依据]的[输入框]。可以[点击]名称为[检索]的[按钮]。可以[点击]名称为[重置]的[按钮]。

In [ ]:
import random
from random import choice
from datetime import datetime
advs = ["", "请"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data = {}
data["审判程序s"] = ["民事一审"]
data["案由s"] = "民事案由 刑事案由 劳动争议 物权纠纷 行政处罚 证券纠纷 财产保全 污染环境罪 侵权责任纠纷 海事海商纠纷 证券投资咨询纠纷 违法司法罚款赔偿".split()
data["文书类型s"] = "判决书 裁定书 决定书 文书 调解书 通知书".split()
data["法院名称s"] = "广东省深圳市中级人民法院 上海市金融法院 北京市第三中级人民法院 湖南省高级人民法院 江苏省高级人民法院 北京市高级人民法院 北京市第三中级人民法院 北京市第一中级人民法院 浙江省杭州市中级人民法院 北京市第三中级人民法院".split()
data["案件类型s"] = "司法制裁案件 管辖案件 行政案件 国家赔偿与司法救助案件 民事案件 非诉保全审查案件 刑事案件 强制清算与破产案件 国际司法协助案件 非诉保全审查案件".split()
data["当事人s"] = "凌孟夏 牛晨轩 马宏茂 陈义 卢嘉云 韦希 郎辉 洪嘉歆".split()
data["律师s"] = "郑奇伟 贺涵易 邱斐 甄俊誉 孔博涛 宗彦 郑琛瑞 喻嘉悦".split()
data["审判人员s"] = "郑念雁 卢明志 焦青易 陈欣悦 宗彦 林映冬 卢木仲 董林".split()
data["审判程序s"] = ["民事提级管辖"]
data["全文检索s"] = "华东科技 中国铁建 中国人寿 中国平安 中国建筑 中国中铁 牧原股份 中国石化 上汽集团 东风汽车集团 中国医药集团 中金公司 中国移动 工商银行 苏宁控股集团 太平洋建设集团".split()
data["years"] = {'五个月': ['2022-10-15', '2023-03-15'], '十个月': ['2022-05-15', '2023-03-15'],'半年': ['2022-09-15', '2023-03-15'], '三年': ['2020-03-15', '2023-03-15'], '二年': ['2021-03-15', '2023-03-15'], '四年': ['2019-03-15', '2023-03-15']}
data["裁判日期_1s"] = "2018-01-01 2019-01-01 2020-01-01 2021-01-01".split()
data["裁判日期_2s"] = "2023-03-01 2023-09-01".split()
def getInstruction():
    wenshu_court_instruction_detail = []
    for i in range(1,2000000):
        adv = choice(advs)
        verb = choice(verbs)
        全文检索 = choice(data["全文检索s"])
        案由 = choice(data["案由s"])
        法院名称 = choice(data["法院名称s"])
        审判人员 = choice(data["审判人员s"])
        案件类型 = choice(data["案件类型s"])
        审判程序 = choice(data["审判程序s"])
        文书类型 = choice(data["文书类型s"])
        year = choice(list(data["years"].keys()))
        val = data["years"][year]
        裁判日期_1 = choice(data["裁判日期_1s"])
        裁判日期_2 = choice(data["裁判日期_2s"])
        当事人 = choice(data["当事人s"])
        律师 = choice(data["律师s"])

        sample = {}
        key_value = {}
        instruct = adv + verb

        key_value["检索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        if(random.random()>0.5):
            key_value["全文检索"] = {'dom_type': '输入框', 'value': 全文检索, 'action': '输入'}
            instruct += 全文检索
        if(random.random()>0.5):
            key_value["当事人"] = {'dom_type': '输入框', 'value': 当事人, 'action': '输入'}
            instruct += "当事人是" + 当事人
        if(random.random()>0.5):
            key_value["律师"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
            instruct += "律师" + 律师
        if(random.random()>0.5):
            key_value["裁判日期_1"] = {'dom_type': '日期输入框', 'value': val[0], 'action': '输入'}
            key_value["裁判日期_2"] = {'dom_type': '日期输入框', 'value': val[1], 'action': '输入'}
            instruct += "最近" + year + "的"
        if(random.random() > 0.5 and year not in instruct):
            key_value["裁判日期_1"] = {'dom_type': '日期输入框', 'value': 裁判日期_1, 'action': '输入'}
            key_value["裁判日期_2"] = {'dom_type': '日期输入框', 'value': 裁判日期_2, 'action': '输入'}
            裁判日期_1 = datetime.strptime(裁判日期_1, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')
            裁判日期_2 = datetime.strptime(裁判日期_2, "%Y-%m-%d").strftime("%Y年%m月%d日").replace('年0', '年').replace('月0', '月')

            instruct += (裁判日期_1 + "到" + 裁判日期_2)

        if(random.random()>0.5):
            key_value["案由"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[案由] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "案由是" + 案由
        if(random.random()>0.5):
            key_value["法院名称"] = {'dom_type': '输入框', 'value': 法院名称, 'action': '输入'}
            instruct += 法院名称

        if(案由 not in instruct and random.random()>0.5):
            key_value["案件类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[案件类型] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += 案件类型
        if(random.random()>0.5 and random.random()>0.5):
            key_value["审判程序"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
            key_value[审判程序] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
            instruct += "审判程序为" + 审判程序

        key_value["文书类型"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[文书类型] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 文书类型
        sample["instruction"] = instruct
        key_value["高级检索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
        sample["key-value"] = key_value
        wenshu_court_instruction_detail.append(sample)
    return random.sample(wenshu_court_instruction_detail, 10000)
res = getInstruction()
print(len(res))
print(res[-30:])



10000
[{'instruction': '检索中国石化律师宗彦最近三年的案由是物权纠纷广东省深圳市中级人民法院审判程序为民事提级管辖裁定书', 'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '全文检索': {'dom_type': '输入框', 'value': '中国石化', 'action': '输入'}, '律师': {'dom_type': '输入框', 'value': '宗彦', 'action': '输入'}, '裁判日期_1': {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}, '裁判日期_2': {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}, '案由': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '物权纠纷': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '法院名称': {'dom_type': '输入框', 'value': '广东省深圳市中级人民法院', 'action': '输入'}, '审判程序': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '民事提级管辖': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '文书类型': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '裁定书': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '高级检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}}}, {'instruction': '查找出中国中铁当事人是陈义律师孔博涛2021年1月1日到2023年9月1日司法制裁案件决定书', 'key-value': {'检索': {'dom_type': '按钮', 'valu

## 学信网 code by guo




##整理网页信息 prompt:
网页[学信网]包括：可以[点击]名称为[所在省市]的[下拉框]，其[下拉框选项]有[北京市 天津市 河北省 山西省 内蒙古自治区 辽宁省 吉林省 黑龙江省 上海市 江苏省 浙江省 安徽省 福建省 江西省 山东省 河南省 湖北省 湖南省 广东省 广西壮族自治区 海南省 重庆市 四川省 贵州省 云南省 西藏自治区 陕西省 甘肃省 青海省 宁夏回族自治区 新疆维吾尔自治区]。可以[输入]名称为[招生单位]的[输入框]。
可以[点击]名称为[门类类别]的[下拉框]，其[下拉框选项]有[哲学 经济学 法学 教育学 文学 历史学 理学 工学 农学 医学 军事学 管理学 艺术学 交叉学科]。可以[点击]名称为[学科类别]的[下拉框]，其[下拉框选项]有[哲学 理论经济学 应用经济学 金融 应用统计 税务 国际商务 保险 资产评估 审计 统计学 法学 政治学 社会学 民族学 马克思主义理论 公安学 法律 社会工作 警务 教育学 心理学 体育学 教育 体育 汉语国际教育 应用心理 中国语言文学 外国语言文学 新闻传播学 翻译 新闻与传播 出版 考古学 中国史 世界史 文物与博物馆 数学 物理学 化学 天文学 地理学 大气科学 海洋科学 地球物理学 地质学 生物学 系统科学 科学技术史 生态学 统计学 心理学 力学 材料科学与工程 电子科学与技术 计算机科学与技术 环境科学与工程 生物医学工程 基础医学 公共卫生与预防医学 药学 中药学 医学技术 护理学 力学 机械工程 光学工程 仪器科学与技术 材料科学与工程 冶金工程 动力工程及工程热物理 电气工程 电子科学与技术 信息与通信工程 控制科学与工程 计算机科学与技术 建筑学 土木工程 水利工程 测绘科学与技术 化学工程与技术 地质资源与地质工程 矿业工程 石油与天然气工程 纺织科学与工程 轻工技术与工程 交通运输工程 船舶与海洋工程 航空宇航科学与技术 兵器科学与技术 核科学与技术 农业工程 林业工程 环境科学与工程 生物医学工程 食品科学与工程 城乡规划学 风景园林学 软件工程 生物工程 安全科学与工程 公安技术 网络空间安全 建筑学 城市规划 电子信息 机械 材料与化工 资源与环境 能源动力 土木水利 生物与医药 交通运输 科学技术史 管理科学与工程 设计学 作物学 园艺学 农业资源与环境 植物保护 畜牧学 兽医学 林学 水产 草学 农业 兽医 风景园林 林业 科学技术史 环境科学与工程 食品科学与工程 风景园林学 基础医学 临床医学 口腔医学 公共卫生与预防医学 中医学 中西医结合 药学 中药学 特种医学 医学技术 护理学 临床医学 口腔医学 公共卫生 护理 药学 中药学 中医 科学技术史 生物医学工程 军事思想及军事历史 战略学 战役学 战术学 军队指挥学 军事管理学 军队政治工作学 军事后勤学 军事装备学 军事训练学 军事 管理科学与工程 工商管理 农林经济管理 公共管理 图书情报与档案管理 工商管理 公共管理 会计 旅游管理 图书情报 工程管理 艺术学理论 音乐与舞蹈学 戏剧与影视学 美术学 设计学 艺术 集成电路科学与工程 国家安全学]。可以[点击]名称为[专业名称]的[下拉框]，其[下拉框选项]有[伦理学 刑法学 教育史 新闻学 档案学 植物学 社会学 统计学 药理学 中国哲学 国际关系 地球化学 工商管理 经济法学 药物化学 计算数学 食品科学 人文地理学 基础兽医学 基础心理学 口腔基础医学 城乡规划管理 区域发展与规划 计算机应用技术 马克思主义哲学]。可以[点击]名称为[学习方式]的[下拉框]，其[下拉框选项]有[全日制 非全日制]。可以[点击]名称为[查询]的[按钮]。

In [ ]:
import random
from random import choice
data ={}
advs = ["", "请","帮"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data["省市s"] = "青海省 河北省 甘肃省 北京市 湖北省 山西省 重庆市 云南省 浙江省 广西壮族自治区 新疆维吾尔自治区 天津市 陕西省 黑龙江省 西藏自治区 湖南省 上海市 内蒙古自治区".split()
data["招生单位s"] = "西安交通大学 大连理工大学 兰州大学 山东大学 华东师范大学 北京师范大学 上海交通大学 华南理工大学 清华大学 华中科技大学 电子科技大学 中国海洋大学 同济大学 厦门大学 哈尔滨工业大学 复旦大学 四川大学 国防科学技术大学 中国农业大学 西北农林科技大学 北京理工大学".split()
data["门类类别s"] = "哲学 经济学 法学 教育学 文学 历史学 理学 工学 农学 医学 军事学 管理学 艺术学 交叉学科 专业学位 专硕".split()
data["学科类别s"] = "计算机应用技术 教育 食品科学 刑法学 档案学 植物学 国际关系 教育史 兽医 统计学 兽医学 地球化学 哲学 化学 基础医学 农业 口腔基础医学 药物化学 城乡规划管理 马克思主义哲学 军事学 计算数学 军事 工商管理 数学 新闻学 社会学 法学 教育学 基础兽医学 艺术".split()
data["专业名称s"] = "国际关系 教育史 计算机应用技术 地球化学 基础兽医学 工商管理 食品科学 口腔基础医学 药物化学 统计学 城乡规划管理 刑法学 档案学 马克思主义哲学 计算数学 植物学 新闻学 社会学".split()
data["学习方式s"] = "全日制 非全日制 ".split()

chsi_instruction_detail = []
for i in range(1, 1000000):
    adv = choice(advs)
    verb = choice(verbs)
    省市 = choice(data["省市s"])
    招生单位 = choice(data["招生单位s"])
    门类类别 = choice(data["门类类别s"])
    学科类别 = choice(data["学科类别s"])
    专业名称 = choice(data["专业名称s"])
    学习方式 = choice(data["学习方式s"])
    sample = {}
    key_value = {}
    instruct = ""

    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct += adv + verb

    if(random.random()>0.5):
      key_value["招生单位"] = {'dom_type': '输入框', 'value': 招生单位, 'action': '输入'}
      instruct += 招生单位 + ","
    if(random.random()>0.5):
      key_value["门类类别"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
      key_value[门类类别] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
      if(门类类别 != "专业学位" and 门类类别 != "专硕"):
          instruct += 门类类别 + "学硕,"
      else:
          instruct += 门类类别 + ","
    if(random.random()>0.5):
        key_value["研究生院"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
        instruct += "招生单位有研究生院,"
    if(random.random()>0.5):
        key_value["博士点"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
        instruct += "招生单位有博士点,"
    if(random.random()>0.5):
        key_value["自划线院校"] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
        instruct += "招生单位自划线院校,"

    if(random.random()>0.5):
      key_value["学科类别"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
      key_value[学科类别] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
      instruct += "学科类别是" + 学科类别 + ","
    if(random.random()>0.5):
      key_value["专业名称"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
      key_value[专业名称] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
      instruct +=  专业名称 + "专业,"
    if(random.random()>0.5):
      key_value["学习方式"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
      key_value[学习方式] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
      instruct += 学习方式 + ","

    instruct += "招生情况"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    chsi_instruction_detail.append(sample)
chsi_instruction_detail = random.sample(chsi_instruction_detail, 10000)
print(len(chsi_instruction_detail))
print(chsi_instruction_detail[-30:])

10000
[{'instruction': '检索一下理学学硕,招生单位有研究生院,招生单位有博士点,学科类别是地球化学,非全日制,招生情况', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '门类类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '理学': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '研究生院': {'dom_type': '复选框', 'value': '', 'action': '点击'}, '博士点': {'dom_type': '复选框', 'value': '', 'action': '点击'}, '学科类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '地球化学': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '学习方式': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '非全日制': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}}, {'instruction': '请查一下华中科技大学,法学学硕,招生单位有研究生院,招生单位有博士点,招生单位自划线院校,学科类别是军事,招生情况', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '招生单位': {'dom_type': '输入框', 'value': '华中科技大学', 'action': '输入'}, '门类类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '法学': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}, '研究生院': {'dom_type': '复选框', 'value': '', 'action': '点击'}, '博士点

## 中国法律服务网 code by guo
### 整理网页信息 prompt:
网页[中国法律服务网]包括：可以[点击]名称为[查询]的[按钮]。可以[点击]名称为[律师]的[按钮]。可以[点击]名称为[律师事务所]的[按钮]。可以[输入]名称为[请输入律所关键字]的[输入框]。可以[点击]名称为[请选择]的[下拉框]，其[下拉框选项]有[上海市 北京市 吉林省 天津市 安徽省 山东省 山西省 广东省 江苏省 江西省 河北省 河南省 浙江省 湖北省 福建省 辽宁省 黑龙江省 内蒙古自治区]。可以[点击]名称为[东城区 丰台区 大兴区 密云区 平谷区 延庆区 怀柔区 房山区 昌平区 朝阳区 海淀区 西城区 通州区 顺义区 石景山区 门头沟区 广西壮族自治区]的[单选框]。


In [ ]:
page = "中国法律服务网"
import random
from random import choice
data ={}
advs = ["", "请","帮"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data["请输入律所关键字s"] = "致敦律师事务所 恒德律师事务所 多禾律师事务所 星权律师事务所 尤斯律师事务所 普光律师事务所 中仲律师事务所 泽仪律师事务所 易行律师事务所 存诚律师事务所 英舜律师事务所 京益律师事务所 金台律师事务所".split()
data["请选择s"] = "安徽省 天津市 北京市 山西省 辽宁省 广东省 江西省 浙江省 吉林省 河南省 江苏省 山东省 福建省 黑龙江省 河北省 上海市 内蒙古自治区".split()
data["单选框s"] = "门头沟区 怀柔区 昌平区 通州区 丰台区 西城区 石景山区 密云区 房山区 朝阳区 顺义区 东城区 延庆区".split()
data["律师s"] = "刘威 卢木仲 陈欣悦 童康裕 陈义 董林 林映冬 洪嘉歆 郎辉 郑念雁".split()



legal_service_instruction_detail = []
for i in range(1, 1000000):
    adv = choice(advs)
    verb = choice(verbs)
    请输入律所关键字 = choice(data["请输入律所关键字s"])
    请选择 = choice(data["请选择s"])
    单选框 = choice(data["单选框s"])
    律师 = choice(data["律师s"])
    sample = {}
    key_value = {}
    instruct = ""
    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct += adv + verb

    if(random.random()>0.5):
        key_value["请选择"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[请选择] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 请选择

    if(random.random()>0.5):
        key_value[单选框] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += 单选框

    if(random.random()>0.5):
        key_value["请输入律所关键字"] = {'dom_type': '输入框', 'value': 请输入律所关键字, 'action': '输入'}
        instruct += 请输入律所关键字

    if(请输入律所关键字 in instruct):
        if(random.random()>0.5):
            key_value["请输入律师关键字"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
            instruct += 律师 + "律师"
    else:
        key_value["请输入律师关键字"] = {'dom_type': '输入框', 'value': 律师, 'action': '输入'}
        instruct += 律师 + "律师"

    instruct += "的信息"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    legal_service_instruction_detail.append(sample)

    sample = {}
    key_value = {}
    instruct = ""
    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct += adv + verb

    if(random.random()>0.5):
        key_value["请选择"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[请选择] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += 请选择

    if(random.random()>0.5):
        key_value[单选框] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
        instruct += 单选框
    key_value["律师"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct += "律师的"
    instruct += "信息"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    #print(sample)
    legal_service_instruction_detail.append(sample)
legal_service_instruction_detail = random.sample(legal_service_instruction_detail, 10000)
print(len(legal_service_instruction_detail))
print(legal_service_instruction_detail[:30])

3672


## 中国银行保险监督管理委员会 by guo
### 整理网页信息 prompt:
网页[中国银行保险监督管理委员会]包括：可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[机构名称]的[输入框]。可以[输入]名称为[机构地址]的[输入框]。可以[输入]名称为[流水号]的[输入框]。可以[点击]名称为[机构类型_1]的[下拉框]，其[下拉框选项]有[集团 控股 财险 人身险 再保险 资产管理 外资代表处 其他]。可以[点击]名称为[机构类型_2]的[下拉框]，其[下拉框选项]有[综合财险公司 专业农险公司 专业车险公司 专业责任险公司 其他 出口信用险公司]。可以[点击]名称为[机构层级]的[下拉框]，其[下拉框选项]有[总公司 分公司 中心支公司 支公司 营业部 营销服务部 省级分公司 专属机构(电销-总) 专属机构(电销-省分)]。可以[点击]名称为[监管机构_1]的[下拉框]，其[下拉框选项]有[北京监管局 天津监管局 河北监管局 山西监管局 内蒙古监管局 辽宁监管局 吉林监管局 黑龙江监管局 上海监管局 江苏监管局 浙江监管局 安徽监管局 福建监管局 江西监管局 山东监管局 河南监管局 湖北监管局 湖南监管局 广东监管局 广西监管局 海南监管局 重庆监管局 四川监管局 贵州监管局 云南监管局 西藏监管局 陕西监管局 甘肃监管局 青海监管局 宁夏监管局 新疆监管局 大连监管局 宁波监管局 厦门监管局 青岛监管局 深圳监管局]。可以[点击]名称为[监管机构_2]的[下拉框]，其[下拉框选项]有[保定银保监分局 三亚银保监分局 江苏银保监局 银保监会 广东银保监局 秦皇岛银保监分局 重庆银保监局 内蒙古银保监局 大连银保监分局 秦皇岛银保监分局 陕西银保监局 秦皇岛银保监分局 四川银保监局 新疆银保监局 厦门银保监分局 深圳银保监分局 北京银保监局 云南银保监局 鞍山银保监分局 邢台银保监分局 湖北银保监局 海南银保监局 云南银保监局 重庆银保监局 青海银保监局 内蒙古银保监局 贵州银保监局 厦门银保监分局 银保监会 吉林银保监分局 邯郸银保监分局 保定银保监分局 浙江银保监局 福州银保监分局 重庆银保监局 新疆银保监局 广州银保监分局 北京银保监局 厦门银保监分局 山西银保监局 辽宁银保监局 河南银保监局 宁夏银保监局 深圳银保监分局 大连银保监分局 长春银保监分局 四平银保监分局 广州银保监分局 海口银保监分局 陕西银保监局 唐山银保监分局 秦皇岛银保监分局 安徽银保监局 四川银保监局 湖南银保监局 上海银保监局 云南银保监局 天津银保监局 海南银保监局 秦皇岛银保监分局 贵州银保监局 黑龙江银保监局 吉林银保监局 四川银保监局 贵州银保监局 四平银保监分局 福州银保监分局 保定银保监分局 沈阳银保监分局 江苏银保监局 厦门银保监分局 内蒙古银保监局 广东银保监局 济南银保监分局 沈阳银保监分局 浙江银保监局 陕西银保监局 辽宁银保监局 石家庄银保监分局 甘肃银保监局 海口银保监分局 厦门银保监分局 重庆银保监局 厦门银保监分局 海口银保监分局 广西银保监局 山西银保监局 辽宁银保监局 银保监会]。

In [ ]:
import random
from random import choice
page_source = "中国银行保险监督管理委员会"

data ={}
advs = ["", "请","帮"]
verbs = ["查找","查询","搜索","检索","查一下","检索一下","查询一下","查一查","检索一下","查找出"]
data["机构类型_1s"] = "人身险 其他 再保险 外资代表处 产险 资产管理 集团 控股".split()
data["机构类型_2s"] = "专业责任险公司 专业车险公司 其他 出口信用险公司 政策性保险公司 专业农险公司".split()
data["机构层级s"] = "分公司 营业部 营销服务部 中心支公司 专属机构(电销-省分) 省级分公司 支公司 专属机构(电销-总) 总公司".split()
data["监管机构_1s"] = "天津银保监局 黑龙江银保监局 江苏银保监局 云南银保监局 广东银保监局 四川银保监局 陕西银保监局 上海银保监局 重庆银保监局 浙江银保监局 广西银保监局 辽宁银保监局 新疆银保监局 海南银保监局 青海银保监局 山西银保监局 吉林银保监局 银保监会 安徽银保监局 北京银保监局 湖北银保监局 贵州银保监局 河北银保监局".split()
data["监管机构_2s"] = "深圳银保监分局 四平银保监分局 天津银保监局 江苏银保监局 湖南银保监局 广东银保监局 四川银保监局 陕西银保监局 邯郸银保监分局 上海银保监局 鞍山银保监分局 重庆银保监局 浙江银保监局 内蒙古银保监局 广西银保监局 广州银保监分局 三亚银保监分局 吉林银保监局 青岛银保监分局 宁夏银保监局 北京银保监局 贵州银保监局 河北银保监局".split()
data["机构地址s"] = "广西壮族自治区南宁市青秀区桂春路13号 湖南省娄底市娄星区月塘街吉泰邦臣大厦A座 吉林省农安县合隆镇合隆街道小北庄585号的机构信息 吉林省农安县合隆镇合隆街道小北庄585号 德华安顾人寿保险有限公司泰安市新泰支公司 浙江省杭州市拱墅区天水街道中山北路中大广场A座7楼 河北省唐山市路北区唐山金融中心1号楼1405-1号 广西壮族自治区南宁市武鸣区城厢镇灵源7队唐家屯110号铺面 湖南省娄底市娄星区月塘街吉泰邦臣大厦A座的机构信息 河北省邯郸市北环路北东邢台村南".split()
data["流水号s"] = "00115012 00009243 00089443 00115028 00070217 00022951 00009244 00115013 00110228".split()
data["机构名称s"] = "华安财产保险股份有限公司梧州中心支公司 横琴人寿保险有限公司珠海市横琴支公司 合众人寿保险股份有限公司宜昌中心支公司枝江营销服务部 富德生命人寿保险股份有限公司武威中心支公司民勤支公司 富德财产保险股份有限公司 德华安顾人寿保险有限公司泰安市新泰支公司 国富人寿保险股份有限公司柳州分公司 合众人寿保险股份有限公司泰安中心支公司东平营销服务部 国富人寿保险股份有限公司柳州市柳江支公司的机构信息 国富人寿保险股份有限公司柳州市柳江支公司 都邦财产保险股份有限公司北仑支公司".split() #输入框


regulatory_instruction_detail = []

for i in range(1,1000000):
    adv = choice(advs)
    verb = choice(verbs)
    机构类型_1 = choice(data["机构类型_1s"])
    机构类型_2 = choice(data["机构类型_2s"])
    机构层级 = choice(data["机构层级s"])
    监管机构_1 = choice(data["监管机构_1s"])
    监管机构_2 = choice(data["监管机构_2s"])
    机构地址 = choice(data["机构地址s"])
    流水号 = choice(data["流水号s"])
    机构名称 = choice(data["机构名称s"])
    sample = {}
    key_value = {}
    instruct = ""

    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
    instruct += adv + verb

    if(random.random()>0.5):
        key_value["监管机构_1"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[监管机构_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "受监管于" + 监管机构_1 + ","
    if(random.random()>0.5):
        key_value["机构类型_1"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[机构类型_1] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "机构类型为" + 机构类型_1 + ","

    if(random.random()>0.5):
        key_value["机构地址"] = {'dom_type': '输入框', 'value': 机构地址, 'action': '输入'}
        instruct += 机构地址
    if(random.random()>0.5):
        key_value["流水号"] = {'dom_type': '输入框', 'value': 流水号, 'action': '输入'}
        instruct += "流水号为"+ 流水号 + ","
    if(random.random()>0.5):
        key_value["机构名称"] = {'dom_type': '输入框', 'value': 机构名称, 'action': '输入'}
        instruct += "机构名为"+ 机构名称 + "机构信息"
    if(random.random()>0.5 and (监管机构_1 in instruct)):
        key_value["监管机构_2"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[监管机构_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "受监管于" + 监管机构_2 + ","

    if(random.random()>0.5 and (机构类型_1 in instruct)):
        key_value["机构类型_2"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[机构类型_2] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "机构类型为" + 机构类型_2 + ","
    if(random.random()>0.5):
        key_value["机构层级"] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
        key_value[机构层级] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
        instruct += "属于" + 机构层级 + ","
    instruct = instruct.strip(",")
    instruct += "的机构信息"
    sample["instruction"] = instruct
    sample["key-value"] = key_value
    # print(sample)
    regulatory_instruction_detail.append(sample)
regulatory_instruction_detail = random.sample(regulatory_instruction_detail, 10000)
print(len(regulatory_instruction_detail))
print(regulatory_instruction_detail[:100])

10000
[{'instruction': '帮检索河北省邯郸市北环路北东邢台村南流水号为00022951,机构名为富德财产保险股份有限公司机构信息的机构信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '机构地址': {'dom_type': '输入框', 'value': '河北省邯郸市北环路北东邢台村南', 'action': '输入'}, '流水号': {'dom_type': '输入框', 'value': '00022951', 'action': '输入'}, '机构名称': {'dom_type': '输入框', 'value': '富德财产保险股份有限公司', 'action': '输入'}}}, {'instruction': '帮查一下机构名为横琴人寿保险有限公司珠海市横琴支公司机构信息属于营业部的机构信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '机构名称': {'dom_type': '输入框', 'value': '横琴人寿保险有限公司珠海市横琴支公司', 'action': '输入'}, '机构层级': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '营业部': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}}, {'instruction': '帮查找受监管于重庆银保监局的机构信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '监管机构_1': {'dom_type': '下拉框', 'value': '', 'action': '点击'}, '重庆银保监局': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}}, {'instruction': '检索机构类型为外资代表处,浙江省杭州市拱墅区天水街道中山北路中大广场A座7楼流水号为00115028,机构名为富德生命人寿保险股份有

## 整理网页信息 prompt:
网页[中国银行保险监督管理委员会]包括：可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[机构名称]的[输入框]。可以[输入]名称为[机构地址]的[输入框]。可以[输入]名称为[流水号]的[输入框]。可以[点击]名称为[机构类型_1]的[下拉框]，其[下拉框选项]有[集团 控股 财险 人身险 再保险 资产管理 外资代表处 其他]。可以[点击]名称为[机构类型_2]的[下拉框]，其[下拉框选项]有[综合财险公司 专业农险公司 专业车险公司 专业责任险公司 其他 出口信用险公司]。可以[点击]名称为[机构层级]的[下拉框]，其[下拉框选项]有[总公司 分公司 中心支公司 支公司 营业部 营销服务部 省级分公司 专属机构(电销-总) 专属机构(电销-省分)]。可以[点击]名称为[监管机构_1]的[下拉框]，其[下拉框选项]有[北京监管局 天津监管局 河北监管局 山西监管局 内蒙古监管局 辽宁监管局 吉林监管局 黑龙江监管局 上海监管局 江苏监管局 浙江监管局 安徽监管局 福建监管局 江西监管局 山东监管局 河南监管局 湖北监管局 湖南监管局 广东监管局 广西监管局 海南监管局 重庆监管局 四川监管局 贵州监管局 云南监管局 西藏监管局 陕西监管局 甘肃监管局 青海监管局 宁夏监管局 新疆监管局 大连监管局 宁波监管局 厦门监管局 青岛监管局 深圳监管局]。可以[点击]名称为[监管机构_2]的[下拉框]，其[下拉框选项]有[保定银保监分局 三亚银保监分局 江苏银保监局 银保监会 广东银保监局 秦皇岛银保监分局 重庆银保监局 内蒙古银保监局 大连银保监分局 秦皇岛银保监分局 陕西银保监局 秦皇岛银保监分局 四川银保监局 新疆银保监局 厦门银保监分局 深圳银保监分局 北京银保监局 云南银保监局 鞍山银保监分局 邢台银保监分局 湖北银保监局 海南银保监局 云南银保监局 重庆银保监局 青海银保监局 内蒙古银保监局 贵州银保监局 厦门银保监分局 银保监会 吉林银保监分局 邯郸银保监分局 保定银保监分局 浙江银保监局 福州银保监分局 重庆银保监局 新疆银保监局 广州银保监分局 北京银保监局 厦门银保监分局 山西银保监局 辽宁银保监局 河南银保监局 宁夏银保监局 深圳银保监分局 大连银保监分局 长春银保监分局 四平银保监分局 广州银保监分局 海口银保监分局 陕西银保监局 唐山银保监分局 秦皇岛银保监分局 安徽银保监局 四川银保监局 湖南银保监局 上海银保监局 云南银保监局 天津银保监局 海南银保监局 秦皇岛银保监分局 贵州银保监局 黑龙江银保监局 吉林银保监局 四川银保监局 贵州银保监局 四平银保监分局 福州银保监分局 保定银保监分局 沈阳银保监分局 江苏银保监局 厦门银保监分局 内蒙古银保监局 广东银保监局 济南银保监分局 沈阳银保监分局 浙江银保监局 陕西银保监局 辽宁银保监局 石家庄银保监分局 甘肃银保监局 海口银保监分局 厦门银保监分局 重庆银保监局 厦门银保监分局 海口银保监分局 广西银保监局 山西银保监局 辽宁银保监局 银保监会]。

##卫健委信用信息网_护 Code by 少春

## 整理网页信息 prompt:
网页[卫健委信用信息网_护]包括：可以[输入]名称为[护士姓名]的[输入框]。可以[输入]名称为[执业地点]的[输入框]。可以[输入]名称为[更新时间_1]的[日期输入框]。可以[输入]名称为[更新时间_2]的[日期输入框]。可以[点击]名称为[性别]的[下拉框]，其[下拉框选项]有[男 女]。

In [ ]:
import random
page_source = '卫健委信用信息网_护'
instructions = []
locations = []
names = []

for i, instruction in enumerate(instruction_trainset[10]['instruction_detail']):
    instruction_str = instruction['instruction']
    instructions.append(instruction_str)
    message = instruction['key-value']
    try:
        location = message["执业地点"]
        locations.append(location)
    except:
        #print("这个实例没有地点")
        pass
    try:
        name = message['护士姓名']
        names.append(name)
    except:
        #print("这个实例没有姓名")
        pass

locations = list(set(locations))

names = list(set(names))
# names.extend(["王德辉", "许庚", "陈雷", "李树华", "李五一", "叶京英", "周亮", "张彬", "李晓明", "樊兆民", "韩东一", "夏衍", "高志强", "龚树生"])

querys_head = ['查询', '请查询', '查找', '请查找', '搜索', '请搜索', '检索', '请检索', '查一下', '请查一下', '检索一下', '请搜索：']
sexs = ['男', '女']

times = ['近半年间', '五个月内', '三年之内', '2019年1月1日到2023年9月1日', '最近三年', '2020年1月1日到2023年9月1日期间', '最近五年', '2020年1月1日到2023年9月1日期间', '2018年1月1日到2023年3月1日期间', '2021年1月1日到2023年9月1日期间', '最近十个月']



instruction_detail_1 = []
for query in querys_head:
            for name in names:
                    # situation 1
                    sample = {}
                    instruct = query + name + '的护士信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['护士姓名']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_1.append(sample)

instruction_detail_2 = []
for query in querys_head:
        for location in locations:
                    # situation 2
                    sample = {}
                    instruct = query + location + '的护士信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['执业地点']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_2.append(sample)

instruction_detail_3 = []
for query in querys_head:
        for location in locations:
            for name in names:
                    # situation 3
                    sample = {}
                    instruct = query + location + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['护士姓名', '执业地点']
                    for action in actions:
                        if action == '护士姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_3.append(sample)

instruction_detail_4 = []
for query in querys_head:
    for sex in sexs:
        for location in locations:
            for name in names:
                    # situation 4
                    sample = {}
                    instruct = query + location + sex + '护士' + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['护士姓名', '性别', '执业地点']
                    for action in actions:
                        if action == '护士姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '性别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_4.append(sample)

instruction_detail_5 = []
for query in querys_head:
        for location in locations:
            for name in names:
                for time in times:
                    # situation 5
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['护士姓名', '执业地点', '更新时间']
                    for action in actions:
                        if action == '护士姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '更新时间为' + time + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_5.append(sample)

instruction_detail_6 = []
for query in querys_head:
    for sex in sexs:
        for location in locations:
            for name in names:
                for time in times:
                    # situation 6
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['护士姓名', '性别', '执业地点', '更新时间']
                    for action in actions:
                        if action == '护士姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '性别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '更新时间为' + time + '的' + sex + '护士' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_6.append(sample)


random.shuffle(instruction_detail_1)
random.shuffle(instruction_detail_2)
random.shuffle(instruction_detail_3)
random.shuffle(instruction_detail_4)
random.shuffle(instruction_detail_5)
random.shuffle(instruction_detail_6)
instruction_detail = []
instruction_detail.extend(instruction_detail_1[:10000])
instruction_detail.extend(instruction_detail_2[:10000])
instruction_detail.extend(instruction_detail_3[:10000])
instruction_detail.extend(instruction_detail_4[:10000])
instruction_detail.extend(instruction_detail_5[:10000])
instruction_detail.extend(instruction_detail_6[:10000])
print(len(instruction_detail))
random.shuffle(instruction_detail)
print("generate all num: %d" % len(instruction_detail))
instruction_detail = random.sample(instruction_detail, 5000)
print(len(instruction_detail))


这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有姓名
这个实例没有地点
37668
[{'instruction': '检索喻嘉悦的护士信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '护士姓名': {'dom_type': '输入框', 'value': '喻嘉悦', 'action': '输入'}}}, {'instruction': '查找凌孟夏的护士信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '护士姓名': {'dom_type': '输入框', 'value': '凌孟夏', 'action': '输入'}}}, {'instruction': '请查一下邱斐的护士信息', 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}, '护士姓名': {'dom_type': '输入框', 'value': '邱斐', 'action': '输入'}}}, {'instruction': '搜索宗彦的护士信息', 'key-value': {'查询': {'dom_type': '按钮', 'value'

In [ ]:
web_element_all = """可以[点击]名称为[查询]的按钮。可以[输入]名称为[护士姓名]的[输入框]。可以[输入]名称为[执业地点]的[输入框]。可以[输入]名称为[更新时间_1]的[日期输入框]。可以[输入]名称为[更新时间_2]的[日期输入框]。可以[点击]名称为[性别]的[下拉框]，其[下拉框选项]有[男 女]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][护士姓名][输入框]。[输入][执业地点][输入框]。[输入][更新时间_1][日期输入框]。[输入][更新时间_2][日期输入框]。[点击][性别][下拉框]，其[下拉框选项]有[男 女]。"""
web_name = '卫健委信用信息网_护'
asset_nurse_add = generate_train_dataset(label_trainset[10]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=500,
                                         rule_num2=250,
                                         rule_num3=500,
                                         common_num=120,
                                         text_max_len=600)


##卫健委信用信息网_医 Code by 少春





## 整理网页信息 prompt:
网页[卫健委信用信息网_医]包括：可以[输入]名称为[医师姓名]的[输入框]。可以[输入]名称为[执业地点]的[输入框]。可以[输入]名称为[更新时间_1]的[日期输入框]。可以[输入]名称为[更新时间_2]的[日期输入框]。可以[点击]名称为[性别]的[下拉框]，其[下拉框选项]有[男 女]。可以[点击]名称为[执业类别]的[下拉框]，其[下拉框选项]有[临床 口腔 中医 公共卫生 中西医结合]。

In [ ]:
page_source = '卫健委信用信息网_医'
instructions = []
locations = []
names = []
categories = []
for i, instruction in enumerate(instruction_trainset[9]['instruction_detail']):
    instruction_str = instruction['instruction']
    instructions.append(instruction_str)
    message = instruction['key-value']
    try:
        location = message["执业地点"]
        locations.append(location)
    except:
        pass
    try:
        name = message['医师姓名']
        names.append(name)
    except:
        pass
    try:
        category = message['执业类别']
        categories.append(category)
    except:
        pass

locations = list(set(locations))
names = list(set(names))
categories = list(set(categories))

locations = list(set(locations))
# locations = locations[:5]
# locations.extend(["北京大学第三医院", "北京协和医院", "中国兵器工业集团公司北京北方医院", "北京铁路局北京铁路分局双桥铁路医院",
#                   "北京天坛医院", "北京中医药大学联科肾病医院", "武汉同济医院", "武汉大学人民医院",
#                   "华中科技大学同济医学院附属同济医院", "中国人民解放军白求恩国际和平医院"])

names = list(set(names))

querys_head = ['查询', '请查询', '查找', '请查找', '搜索', '请搜索', '检索', '请检索', '查一下', '请查一下', '检索一下', '请搜索：']
sexs = ['男', '女']
times = ['近半年间', '五个月内', '三年之内', '2019年1月1日到2023年9月1日', '最近三年', '2020年1月1日到2023年9月1日期间', '最近五年', '2020年1月1日到2023年9月1日期间', '2018年1月1日到2023年3月1日期间', '2021年1月1日到2023年9月1日期间', '最近十个月']

instruction_detail_1 = []
for query in querys_head:
            for name in names:
                    # situation 1
                    sample = {}
                    instruct = query + name + '的医师信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_1.append(sample)

instruction_detail_2 = []
for query in querys_head:
        for location in locations:
                    # situation 2
                    sample = {}
                    instruct = query + location + '的医师信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['执业地点']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_2.append(sample)

instruction_detail_3 = []
for query in querys_head:
        for location in locations:
            for name in names:
                    # situation 3
                    sample = {}
                    instruct = query + location + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '执业地点']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_3.append(sample)

instruction_detail_4 = []
for query in querys_head:
    for sex in sexs:
        for location in locations:
            for name in names:
                    # situation 4
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '性别', '执业地点']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '性别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}

                    sample = {}
                    instruct = query + location + '的' + sex + '医师' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_4.append(sample)

print("?")
instruction_detail_5 = []
for query in querys_head:
        for location in locations:
            for name in names:
                for time in times:
                    # situation 5
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '执业地点', '更新时间']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '更新时间为' + time + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_5.append(sample)

print("??")
instruction_detail_6 = []
for query in querys_head:
    for sex in sexs:
        for location in locations:
            for name in names:
                for time in times:
                    # situation 6
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '性别', '执业地点', '更新时间']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '性别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '更新时间为' + time + '的' + sex + '医师' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_6.append(sample)

print("???")
instruction_detail_7 = []
for query in querys_head:
    for category in categories:
                    # situation 7
                    sample = {}
                    instruct = query + '执业类别为' + category + '的医师信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['执业类别']
                    for action in actions:
                        key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                        key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                    sample["key-value"] = key_value
                    instruction_detail_7.append(sample)

print("????")
instruction_detail_8 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for category in categories:
                    # situation 8
                    sample = {}
                    instruct = query + location + '执业类别为' + category + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '执业类别', '执业地点']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '执业类别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_8.append(sample)

instruction_detail_9 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for sex in sexs:
                for category in categories:
                    # situation 9
                    sample = {}
                    instruct = query + location + '执业类别为' + category + '的' + sex + '医师' + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '性别', '执业类别', '执业地点']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '性别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        elif action == '执业类别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        else:
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_9.append(sample)

print("??????")
instruction_detail_10 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for time in times:
                for category in categories:
                    # situation 10
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '执业类别', '执业地点', '更新时间']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '执业类别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '更新时间为' + time + '的执业类别为' + category + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_10.append(sample)


print("???????")
instruction_detail_11 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for sex in sexs:
                for time in times:
                    for category in categories:
                        # situation 11
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['医师姓名', '性别', '执业类别', '执业地点', '更新时间']
                        for action in actions:
                            if action == '医师姓名':
                                key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                            elif action == '性别':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action == '执业类别':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action == '执业地点':
                                key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                            else:
                                if time == '近半年间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '五个月内':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time in ['三年之内', '最近三年']:
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '2019年1月1日到2023年9月1日':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '2020年1月1日到2023年9月1日期间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '最近五年':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '2020年1月1日到2023年9月1日期间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '最近十个月':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                        sample = {}
                        instruct = query + location +'更新时间为' + time + '的执业类别为' + category + '的' + sex + '医师' + name + '的信息'
                        sample["instruction"] = instruct
                        sample["key-value"] = key_value
                        instruction_detail_11.append(sample)



print("???????")
instruction_detail_12 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for time in times:
                for category in categories:
                    # situation 12
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['医师姓名', '执业类别', '执业地点', '更新时间']
                    for action in actions:
                        if action == '医师姓名':
                            key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                        elif action == '执业类别':
                            key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                            key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        elif action == '执业地点':
                            key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                        else:
                            if time == '近半年间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '五个月内':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time in ['三年之内', '最近三年']:
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2019年1月1日到2023年9月1日':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近五年':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                            elif time == '2020年1月1日到2023年9月1日期间':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                            elif time == '最近十个月':
                                key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                    sample = {}
                    instruct = query + location + '执业类别为' + category + '的更新时间为' + time + '的' + name + '的信息'
                    sample["instruction"] = instruct
                    sample["key-value"] = key_value
                    instruction_detail_12.append(sample)


print("???????")
instruction_detail_13 = []
for query in querys_head:
    for location in locations:
        for name in names:
            for sex in sexs:
                for time in times:
                    for category in categories:
                        # situation 13
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['医师姓名', '性别', '执业类别', '执业地点', '更新时间']
                        for action in actions:
                            if action == '医师姓名':
                                key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                            elif action == '性别':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[sex] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action == '执业类别':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[category] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action == '执业地点':
                                key_value[action] = {'dom_type': '输入框', 'value': location, 'action': '输入'}
                            else:
                                if time == '近半年间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-09-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '五个月内':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2022-10-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time in ['三年之内', '最近三年']:
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-03-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '2019年1月1日到2023年9月1日':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2019-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '2020年1月1日到2023年9月1日期间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '最近五年':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-03-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}
                                elif time == '2020年1月1日到2023年9月1日期间':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2020-01-01', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-09-01', 'action': '输入'}
                                elif time == '最近十个月':
                                    key_value['更新时间_1'] = {'dom_type': '日期输入框', 'value': '2018-08-15', 'action': '输入'}
                                    key_value['更新时间_2'] = {'dom_type': '日期输入框', 'value': '2023-03-15', 'action': '输入'}

                        sample = {}
                        instruct = query + location + '执业类别为' + category + '的更新时间为' + time + '的' + sex + '医师' + name + '的信息'
                        sample["instruction"] = instruct
                        sample["key-value"] = key_value
                        instruction_detail_13.append(sample)

print("all end!!!")
random.shuffle(instruction_detail_1)
random.shuffle(instruction_detail_2)
random.shuffle(instruction_detail_3)
random.shuffle(instruction_detail_4)
random.shuffle(instruction_detail_5)
random.shuffle(instruction_detail_6)
random.shuffle(instruction_detail_7)
random.shuffle(instruction_detail_8)
random.shuffle(instruction_detail_9)
random.shuffle(instruction_detail_10)
random.shuffle(instruction_detail_11)
random.shuffle(instruction_detail_12)
random.shuffle(instruction_detail_13)
instruction_detail = []
instruction_detail.extend(instruction_detail_1[:5000])
instruction_detail.extend(instruction_detail_2[:5000])
instruction_detail.extend(instruction_detail_3[:5000])
instruction_detail.extend(instruction_detail_4[:5000])
instruction_detail.extend(instruction_detail_5[:5000])
instruction_detail.extend(instruction_detail_6[:5000])
instruction_detail.extend(instruction_detail_7[:5000])
instruction_detail.extend(instruction_detail_8[:5000])
instruction_detail.extend(instruction_detail_9[:5000])
instruction_detail.extend(instruction_detail_10[:5000])
instruction_detail.extend(instruction_detail_11[:5000])
instruction_detail.extend(instruction_detail_12[:5000])
instruction_detail.extend(instruction_detail_13[:5000])

print(len(instruction_detail))
random.shuffle(instruction_detail)
print("generate all num: %d" % len(instruction_detail))
instruction_detail = random.sample(instruction_detail, 5000)
print(len(instruction_detail))

# print(len(label_trainset[9]["instruction_detail"]))
# label_trainset[9]["instruction_detail"].extend(instruction_detail)
# print(len(label_trainset[9]["instruction_detail"]))
web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[医师姓名]的[输入框]。可以[输入]名称为[执业地点]的[输入框]。可以[输入]名称为[更新时间_1]的[日期输入框]。可以[输入]名称为[更新时间_2]的[日期输入框]。可以[点击]名称为[性别]的[下拉框]，其[下拉框选项]有[男 女]。可以[点击]名称为[执业类别]的[下拉框]，其[下拉框选项]有[临床 口腔 中医 公共卫生 中西医结合]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][医师姓名][输入框]。[输入][执业地点][输入框]。[输入][更新时间_1][日期输入框]。[输入][更新时间_2][日期输入框]。[点击][性别][下拉框]，其[下拉框选项]有[男 女]。[点击][执业类别][下拉框]，其[下拉框选项]有[临床 口腔 中医 公共卫生 中西医结合]。"""
web_name = '卫健委信用信息网_医'
asset_doctor_add = generate_train_dataset(label_trainset[9]['instruction_detail'],
                                          instruction_detail,
                                          web_name,
                                          web_element_all_optimize,
                                          rule_num1=1000,
                                          rule_num2=500,
                                          rule_num3=1000,
                                          common_num=250,
                                          text_max_len=600)

这个实例没有姓名
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有类别
这个实例没有地点
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有地点
这个实例没有类别
这个实例没有地点
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有姓名
这个实例没有类别
这个实例没有地点
这个实例没有姓名
这个实例没有姓名
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别
这个实例没有类别


{'instruction': '请搜索：甄俊誉的医师信息',
 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
  '医师姓名': {'dom_type': '输入框', 'value': '甄俊誉', 'action': '输入'}}}

In [ ]:
web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[医师姓名]的[输入框]。可以[输入]名称为[执业地点]的[输入框]。可以[输入]名称为[更新时间_1]的[日期输入框]。可以[输入]名称为[更新时间_2]的[日期输入框]。可以[点击]名称为[性别]的[下拉框]，其[下拉框选项]有[男 女]。可以[点击]名称为[执业类别]的[下拉框]，其[下拉框选项]有[临床 口腔 中医 公共卫生 中西医结合]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][医师姓名][输入框]。[输入][执业地点][输入框]。[输入][更新时间_1][日期输入框]。[输入][更新时间_2][日期输入框]。[点击][性别][下拉框]，其[下拉框选项]有[男 女]。[点击][执业类别][下拉框]，其[下拉框选项]有[临床 口腔 中医 公共卫生 中西医结合]。"""
web_name = '卫健委信用信息网_医'
asset_doctor_add = generate_train_dataset(label_trainset[9]['instruction_detail'],
                                          instruction_detail,
                                          web_name,
                                          web_element_all_optimize,
                                          rule_num1=1000,
                                          rule_num2=500,
                                          rule_num3=1000,
                                          common_num=250,
                                          text_max_len=600)

## 顺丰快递查收寄标准 Code by 少春

## 整理网页信息 prompt:
网页[顺丰快递查收寄标准]包括：可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[物品名称]的[输入框]。可以[点击]名称为[公司_1]的[单选框]。可以[点击]名称为[公司_2]的[单选框]。可以[点击]名称为[个人_1]的[单选框]。可以[点击]名称为[个人_2]的[单选框]。

In [ ]:
import json
import random

LABEL_PATH = '/content/drive/MyDrive/2023S-T1-A-Data/'
label_trainset = json.load(open(LABEL_PATH + 'label_trainset.json', encoding='utf8'))
instruction_trainset = json.load(open(LABEL_PATH + 'instruction_trainset.json', encoding='utf8'))
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json', encoding='utf8'))

page_source = '顺丰快递查收寄标准'
instructions = []
locations_1 = []
locations_2 = []
things = []
for i, instruction in enumerate(instruction_trainset[18]['instruction_detail']):
    instruction_str = instruction['instruction']
    instructions.append(instruction_str)
    message = instruction['key-value']
    try:
        location = message["始发地"]
        locations_1.append(location)
    except:
        print("这个实例没有始发地")
    try:
        location = message["目的地"]
        locations_2.append(location)
    except:
        print("这个实例没有目的地")
    try:
        thing = message['物品名称']
        things.append(thing)
    except:
        print("这个实例没有物品")

locations_1 = list(set(locations_1))
locations_2 = list(set(locations_2))
things = list(set(things))
querys_head = ['查一下是否可以', '请查一下是否可以', '请查询是否可以', '请检索是否可以', '检索一下是否可以', '查一查是否可以', '帮我查一下是否可以', '查询是否可以']
names_1 = ['个人_1', '公司_1']
names_2 = ['个人_2', '公司_2']
querys_mid = ['寄往', '寄到']
instruction_detail_1 = []
for query_1 in querys_head:
    for query_2 in querys_mid:
        for location_1 in locations_1:
            for location_2 in locations_2:
                for thing in things:
                    for name_1 in names_1:
                        for name_2 in names_2:
                            # situation 1
                            location_11 = ''.join(location_1.split('/'))
                            location_22 = ''.join(location_2.split('/'))
                            name_11 = name_1.split('_1')[0]
                            name_22 = name_2.split('_2')[0]
                            sample = {}
                            instruct = query_1 + '把' + thing + '从' + location_11 + query_2 + location_22 + '，其中寄件人身份是' + name_11 + '，收件人身份是' + name_22
                            sample["instruction"] = instruct
                            key_value = {}
                            key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                            actions = ['始发地', '目的地', 'name', '物品名称']
                            for action in actions:
                                if action == '始发地':
                                    key_value[action] = {'dom_type': '输入框', 'value': location_1, 'action': '输入'}
                                elif action == '目的地':
                                    key_value[action] = {'dom_type': '输入框', 'value': location_2, 'action': '输入'}
                                elif action == 'name':
                                    key_value[name_1] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                                    key_value[name_2] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                                else:
                                    key_value[action] = {'dom_type': '输入框', 'value': thing, 'action': '输入'}
                            sample["key-value"] = key_value
                            instruction_detail_1.append(sample)

querys_head = ['请查询能否以', '帮我查一下能否以', '查一查能否以', '查一下能否以', '检索一下能否以', '查询能否以', '请查一下能否以', '请检索能否以']
instruction_detail_2 = []
for query_1 in querys_head:
    for query_2 in querys_mid:
        for location_1 in locations_1:
            for location_2 in locations_2:
                for thing in things:
                    for name_1 in names_1:
                        for name_2 in names_2:
                            # situation 1
                            location_11 = ''.join(location_1.split('/'))
                            location_22 = ''.join(location_2.split('/'))
                            name_11 = name_1.split('_1')[0]
                            name_22 = name_2.split('_2')[0]
                            sample = {}
                            instruct = query_1 + name_11 + '身份把' + thing + '从' + location_11 + query_2 + location_22 + '，其中收件人身份是' + name_22
                            sample["instruction"] = instruct
                            key_value = {}
                            key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                            actions = ['始发地', '目的地', 'name', '物品名称']
                            for action in actions:
                                if action == '始发地':
                                    key_value[action] = {'dom_type': '输入框', 'value': location_1, 'action': '输入'}
                                elif action == '目的地':
                                    key_value[action] = {'dom_type': '输入框', 'value': location_2, 'action': '输入'}
                                elif action == 'name':
                                    key_value[name_1] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                                    key_value[name_2] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                                else:
                                    key_value[action] = {'dom_type': '输入框', 'value': thing, 'action': '输入'}
                            sample["key-value"] = key_value
                            instruction_detail_2.append(sample)
random.shuffle(instruction_detail_1)
random.shuffle(instruction_detail_2)
instruction_detail = []
instruction_detail.extend(instruction_detail_1[:20000])
instruction_detail.extend(instruction_detail_2[:20000])
# instruction_detail[10]

web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[物品名称]的[输入框]。可以[点击]名称为[公司_1]的[单选框]。可以[点击]名称为[公司_2]的[单选框]。可以[点击]名称为[个人_1]的[单选框]。可以[点击]名称为[个人_2]的[单选框]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][始发地][输入框]。[输入][目的地][输入框]。[输入][物品名称][输入框]。[点击][公司_1][单选框]。[点击][公司_2][单选框]。[点击][个人_1][单选框]。[点击][个人_2][单选框]。"""
web_name = '顺丰快递查收寄标准'
asset_sf2_add = generate_train_dataset(label_trainset[18]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=500,
                                         rule_num2=250,
                                         rule_num3=500,
                                         common_num=120,
                                         text_max_len=600)

{'instruction': '查一查是否可以把挂饰从广东省广州市番禺区寄往阿根廷Fanyu，其中寄件人身份是公司，收件人身份是个人',
 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
  '始发地': {'dom_type': '输入框', 'value': '广东省/广州市/番禺区', 'action': '输入'},
  '目的地': {'dom_type': '输入框', 'value': '阿根廷/Fanyu', 'action': '输入'},
  '公司_1': {'dom_type': '单选框', 'value': '', 'action': '点击'},
  '个人_2': {'dom_type': '单选框', 'value': '', 'action': '点击'},
  '物品名称': {'dom_type': '输入框', 'value': '挂饰', 'action': '输入'}}}

In [ ]:
web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[物品名称]的[输入框]。可以[点击]名称为[公司_1]的[单选框]。可以[点击]名称为[公司_2]的[单选框]。可以[点击]名称为[个人_1]的[单选框]。可以[点击]名称为[个人_2]的[单选框]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][始发地][输入框]。[输入][目的地][输入框]。[输入][物品名称][输入框]。[点击][公司_1][单选框]。[点击][公司_2][单选框]。[点击][个人_1][单选框]。[点击][个人_2][单选框]。"""
web_name = '顺丰快递查收寄标准'
asset_sf2_add = generate_train_dataset(label_trainset[18]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=500,
                                         rule_num2=250,
                                         rule_num3=500,
                                         common_num=120,
                                         text_max_len=600)

##顺丰快递查运费时效 code by 少春


## 整理网页信息 prompt:
网页[顺丰快递查运费时效]包括：可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[寄件时间]的[输入框]。可以[输入]名称为[目的地邮编]的[输入框]。可以[输入]名称为[包裹重量]的[输入框]。

In [ ]:
import json
import random
LABEL_PATH = '/content/drive/MyDrive/2023S-T1-A-Data/'
label_trainset = json.load(open(LABEL_PATH + 'label_trainset.json', encoding='utf8'))
instruction_trainset = json.load(open(LABEL_PATH + 'instruction_trainset.json', encoding='utf8'))
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json', encoding='utf8'))
page_source = '顺丰快递查运费时效'
instructions = []
locations_1 = []
locations_2 = []
post_codes = []
weights = []
times = []
for i, instruction in enumerate(instruction_trainset[19]['instruction_detail']):
    instruction_str = instruction['instruction']
    instructions.append(instruction_str)
    message = instruction['key-value']
    try:
        location = message["始发地"]
        locations_1.append(location)
    except:
        print("这个实例没有始发地")
    try:
        location = message["目的地"]
        locations_2.append(location)
    except:
        print("这个实例没有目的地")
    try:
        time = message['寄件时间']
        times.append(time)
    except:
        print("这个实例没有物品")
    try:
        post_code = message['目的地邮编']
        post_codes.append(post_code)
    except:
        print("这个实例没有物品")
    try:
        weight = message['包裹重量']
        weights.append(weight)
    except:
        print("这个实例没有物品")
locations_1 = list(set(locations_1))
locations_2 = list(set(locations_2))
times = list(set(times))
post_codes = list(set(post_codes))
weights = list(set(weights))
querys_head = ['请检索', '请查找', '查找一下', '检索一下', '查一下', '请查一下', '查询', '请查询', '请搜索：']
querys_mid = ['发往', '寄到']
querys_tail = ['快递时效', '快递运费']
querys_tail_2 = ['其中', '']
post_codes = ['999013', '999017', '999026', '999142', '999038', '999148', '999019', '999012', '999025', '999020', '999022', '999024', '999005', '999023', '999006', '999021', '999004', '999014', '999018']
instruction_detail_1 = []
for query_1 in querys_head:
    for query_2 in querys_mid:
        for query_3 in querys_tail:
            for query_4 in querys_tail_2:
                for location_1 in locations_1:
                    for location_2 in locations_2:
                        for weight in weights:
                            for time in times:
                                    # situation 1
                                    location_11 = ''.join(location_1.split('-'))
                                    location_22 = ''.join(location_2.split('-'))
                                    if int(time[11:13]) >= 12:
                                        time_1 = time.replace(' ', '下午')
                                    else:
                                        time_1 = time.replace(' ', '上午')
                                    sample = {}
                                    instruct = query_1 + '从' + location_11 + query_2 + location_22 + '的' + query_3 + \
                                               '，' + query_4 + '包裹重量为' + str(float(weight)) + 'kg，寄件时间是' + time_1
                                    sample["instruction"] = instruct
                                    key_value = {}
                                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                                    actions = ['始发地', '目的地', '寄件时间', '包裹重量']
                                    for action in actions:
                                        if action == '始发地':
                                            key_value[action] = {'dom_type': '输入框', 'value': location_1, 'action': '输入'}
                                        elif action == '目的地':
                                            key_value[action] = {'dom_type': '输入框', 'value': location_2, 'action': '输入'}
                                        elif action == '寄件时间':
                                            key_value[action] = {'dom_type': '输入框', 'value': time, 'action': '输入'}
                                        else:
                                            key_value[action] = {'dom_type': '输入框', 'value': weight, 'action': '输入'}
                                    sample["key-value"] = key_value
                                    instruction_detail_1.append(sample)
instruction_detail_2 = []
for query_1 in querys_head:
    for query_2 in querys_mid:
        for query_3 in querys_tail:
            for query_4 in querys_tail_2:
                for location_1 in locations_1:
                    for location_2 in locations_2:
                        for weight in weights:
                            for time in times:
                                    # situation 2
                                    location_11 = ''.join(location_1.split('-'))
                                    location_22 = ''.join(location_2.split('-'))
                                    if int(time[11:13]) >= 12:
                                        time_1 = time.replace(' ', '下午')
                                    else:
                                        time_1 = time.replace(' ', '上午')
                                    sample = {}
                                    instruct = query_1 + '从' + location_11 + query_2 + location_22 + '的' + query_3 + \
                                               '，' + query_4 + '寄件时间是' + time_1 + '，包裹重量为' + str(float(weight)) + 'kg'
                                    sample["instruction"] = instruct
                                    key_value = {}
                                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                                    actions = ['始发地', '目的地', '寄件时间', '包裹重量']
                                    for action in actions:
                                        if action == '始发地':
                                            key_value[action] = {'dom_type': '输入框', 'value': location_1, 'action': '输入'}
                                        elif action == '目的地':
                                            key_value[action] = {'dom_type': '输入框', 'value': location_2, 'action': '输入'}
                                        elif action == '寄件时间':
                                            key_value[action] = {'dom_type': '输入框', 'value': time, 'action': '输入'}
                                        else:
                                            key_value[action] = {'dom_type': '输入框', 'value': weight, 'action': '输入'}
                                    sample["key-value"] = key_value
                                    instruction_detail_2.append(sample)
random.shuffle(instruction_detail_1)
random.shuffle(instruction_detail_2)
instruction_detail_3 = instruction_detail_2[20000:40000]
for i in range(20000):
        post_code = random.sample(post_codes, 1)
        instruction_detail_3[i]['instruction'] = instruction_detail_3[i]['instruction'] + '，目的地邮编为' + post_code
        instruction_detail_3[i]['key-value']['目的地邮编'] = {'dom_type': '输入框', 'value': post_code, 'action': '输入'}
instruction_detail = []
instruction_detail.extend(instruction_detail_1[:5000])
instruction_detail.extend(instruction_detail_2[:5000])
instruction_detail.extend(instruction_detail_3[:5000])
print(len(instruction_detail))
instruction_detail[10]

web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[寄件时间]的[输入框]。可以[输入]名称为[目的地邮编]的[输入框]。可以[输入]名称为[包裹重量]的[输入框]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][始发地][输入框]。[输入][目的地][输入框]。[输入][寄件时间][输入框]。[输入][目的地邮编][输入框]。[输入][包裹重量][输入框]。"""
web_name = '顺丰快递查运费时效'
asset_sf1_add = generate_train_dataset(label_trainset[19]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

FileNotFoundError: ignored

In [ ]:
web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[始发地]的[输入框]。可以[输入]名称为[目的地]的[输入框]。可以[输入]名称为[寄件时间]的[输入框]。可以[输入]名称为[目的地邮编]的[输入框]。可以[输入]名称为[包裹重量]的[输入框]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][始发地][输入框]。[输入][目的地][输入框]。[输入][寄件时间][输入框]。[输入][目的地邮编][输入框]。[输入][包裹重量][输入框]。"""
web_name = '顺丰快递查运费时效'
asset_sf1_add = generate_train_dataset(label_trainset[19]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

## 安居客 Code by GL

网页[安居客]包括：可以[输入]名称为[小区名称]的[输入框]。可以[点击]名称为[不限 浦东 闵行 松江 宝山 嘉定 徐汇 青浦 静安 普陀 杨浦 奉贤 黄浦 虹口 长宁 金山 崇明 上海周边]的[单选框]。可以[点击]名称为[1号线 2号线 3号线 4号线 5号线 6号线 7号线 8号线 9号线 10号线 11号线 12号线 13号线 14号线 15号线 16号线 17号线 18号线 磁悬浮 浦江线]的[单选框]。可以[点击]名称为[100万以下 100-200万 200-250万 250-300万 300-350万 350-400万 400-500万 500-800万 800-1000万]的[单选框]。可以[点击]名称为[不限 50㎡以下 50-60㎡ 60-70㎡ 70-80㎡ 80-90㎡ 90-100㎡ 100-120㎡ 120-140㎡ 140-210㎡ 210㎡以上]的[单选框]。可以[点击]名称为[房型]的[单选框]，元素为[不限 一室 二室 三室 四室 五室 五室以上]。可以[点击]名称为[特色]的单选框，元素为[近地铁 满五 满二]。可以[点击]名称为[朝向]的[下拉框]，元素为[不限 东 南 西 北 南北 东西 东南 西南 东北 西北]。可以[点击]名称[楼层]的[复选框]元素为[不限 底层 低层 中层 高层 顶层]。可以[点击]名称[年限]的下拉框，[下拉框选项]为[不限 2年内 2-5年 5-10年 10年以上]。可以[点击]名称[类型]的[下拉框]，[下拉框选项]为[不限 老公房 公寓 新里洋房 普通住宅 别墅 其它 平房]。可以[点击]名称为[产权]的[下拉框]，[下拉框选项]为[不限 商品房住宅 商住两用 经济适用房 公房 动迁配套房 其他]。可以[点击]名称为[装修]的[下拉框]，[下拉框选项]为为[不限 毛坯 普通装修 精装修 豪华装修]。

In [ ]:
# import random
# import re


# def encode_test_instruct(web_name, instruct, web_element=""):
#     desc = "用户浏览的网页是[%s]，%s请回答执行什么操作能完成[%s]?" % (web_name, web_element, instruct)
#     return desc


# def encode_test_response(key_values):
#     desc = "在网页上"
#     for key, value in key_values.items():
#         action = ""
#         # print(key, value)
#         if value["action"] == "点击":
#             action = "[点击]名称为[%s]的[%s]" % (key, value["dom_type"])
#         elif value["action"] == "输入":
#             action = "[输入]值[%s]进名称为[%s]的[%s]" % (value["value"], key, value["dom_type"])
#         desc += action + ";"
#     return desc


# def decode_response(desc):
#     response = {}
#     # actions = desc.split("。")[1]
#     regex = r"(\[.*?\])"
#     for action in desc.split(";"):
#         matches = re.findall(regex, action)
#         if len(matches) == 4:
#             response[matches[2][1:-1]] = {'dom_type': matches[3][1:-1],
#                                           'value': matches[1][1:-1],
#                                           'action': '输入'}
#         elif len(matches) == 3:
#             response[matches[1][1:-1]] = {'dom_type': matches[2][1:-1],
#                                           'value': '',
#                                           'action': '点击'}
#     return response


# def trans_key_values2web_element(key_values):
#     desc = ""
#     for key, value in key_values.items():
#         action = ""
#         # print(key, value)
#         if value["action"] == "点击":
#             action = "可以[点击]名称为[%s]的[%s]" % (key, value["dom_type"])
#         elif value["action"] == "输入":
#             action = "可以[输入]名称为[%s]的[%s]" % (key, value["dom_type"])
#         desc += action + "。"
#     return desc


# def encode_train_instruct1(instruct, web_element_all):  # 实现构建样本中的情况1
#     desc = "用户浏览网页包括的全部元素有:%s请先回答这是什么网页？再回答执行什么操作能完成[%s]？" % (
#         web_element_all, instruct)
#     return desc


# def encode_train_instruct2(instruct, web_element_part):  # 实现构建样本中的情况2
#     desc = "用户浏览网页包括的部分元素有:%s请先回答这是什么网页？再回答执行什么操作能完成[%s]？" % (
#         web_element_part, instruct)
#     return desc


# def encode_train_response(web_name, key_value):
#     desc = "用户浏览的网页是[%s]，" % web_name
#     desc += encode_test_response(key_value)
#     return desc


# # 全部网页信息
# def all_ele():
#     web_ele = "网页[安居客]包括：可以[输入]名称为[小区名称]的[输入框]。可以[点击]名称为[不限 浦东 闵行 松江 宝山 嘉定 徐汇 青浦 静安 普陀 杨浦 奉贤 黄浦 虹口 长宁 金山 崇明 上海周边]的[" \
#               "单选框]。可以[点击]名称为[1号线 2号线 3号线 4号线 5号线 6号线 7号线 8号线 9号线 10号线 11号线 12号线 13号线 14号线 15号线 16号线 17号线 18号线 磁悬浮 " \
#               "浦江线]的[单选框]。可以[点击]名称为[100万以下 100-200万 200-250万 250-300万 300-350万 350-400万 400-500万 500-800万 " \
#               "800-1000万]的[单选框]。可以[点击]名称为[不限 50㎡以下 50-60㎡ 60-70㎡ 70-80㎡ 80-90㎡ 90-100㎡ 100-120㎡ 120-140㎡ 140-210㎡ " \
#               "210㎡以上]的[单选框]。可以[点击]名称为[房型]的[单选框]，元素为[不限 一室 二室 三室 四室 五室 五室以上]。可以[点击]名称为[特色]的单选框，元素为[近地铁 满五 满二]。可以[" \
#               "点击]名称为[朝向]的[下拉框]，元素为[不限 东 南 西 北 南北 东西 东南 西南 东北 西北]。可以[点击]名称[楼层]的[复选框]元素为[不限 底层 低层 中层 高层 顶层]。可以[点击]名称[" \
#               "年限]的下拉框，[下拉框选项]为[不限 2年内 2-5年 5-10年 10年以上]。可以[点击]名称[类型]的[下拉框]，[下拉框选项]为[不限 老公房 公寓 新里洋房 普通住宅 别墅 其它 " \
#               "平房]。可以[点击]名称为[产权]的[下拉框]，[下拉框选项]为[不限 商品房住宅 商住两用 经济适用房 公房 动迁配套房 其他]。可以[点击]名称为[装修]的[下拉框]，[下拉框选项]为为[不限 毛坯 " \
#               "普通装修 精装修 豪华装修]。 "
#     advs = ["", "请", "帮助", "我要"]
#     verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]

#     # 输入框
#     input_name = "汤臣一品 广洋院 王家堂小区".split()
#     # 单选框
#     regions = "不限 浦东 闵行 松江 宝山 嘉定 徐汇 青浦 静安 普陀 杨浦 奉贤 黄浦 虹口 长宁 金山 崇明 上海周边".split()
#     region2s = "1号线 2号线 3号线 4号线 5号线 6号线 7号线 8号线 9号线 10号线 11号线 12号线 13号线 14号线 15号线 16号线 17号线 18号线 磁悬浮 浦江线".split()
#     prices = "100万以下 100-200万 200-250万 250-300万 300-350万 350-400万 400-500万 500-800万 800-1000万".split()
#     areas = "不限 50㎡以下 50-60㎡ 60-70㎡ 70-80㎡ 80-90㎡ 90-100㎡ 100-120㎡ 120-140㎡ 140-210㎡ 210㎡以上".split()
#     house_classes = "不限 一室 二室 三室 四室 五室 五室以上".split()
#     # 复选框
#     characters = "近地铁 满五 满二".split()
#     # 下拉框
#     more1 = "不限 东 南 西 北 南北 东西 东南 西南 东北 西北".split()
#     more2 = "不限 底层 低层 中层 高层 顶层".split()
#     more3 = "不限 2年内 2-5年 5-10年 10年以上".split()
#     more4 = "不限 老公房 公寓 新里洋房 普通住宅 别墅 其它 平房".split()
#     more5 = "不限 商品房住宅 商住两用 经济适用房 公房 动迁配套房 其他".split()
#     more6 = "不限 毛坯 普通装修 精装修 豪华装修".split()

#     instruction_detail = []
#     for adv in advs:
#         for verb in verbs:
#             for name in input_name:
#                 for price in prices:
#                     for area in areas:
#                         for house_class in house_classes:
#                             sample = {}
#                             key_value = {}
#                             num_elements = random.randint(0, 3)
#                             elements = random.choices(characters, k=num_elements)
#                             character = ''.join(str(e) for e in elements)
#                             instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + character + '的房子'
#                             key_value['小区名字'] = {'dom_type': '输入框', 'value': name, 'action': '点击'}
#                             key_value[price] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
#                             key_value[area] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
#                             key_value[house_class] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
#                             if len(elements) > 0:
#                                 for ele in elements:
#                                     key_value[ele] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
#                             key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
#                             sample["key-value"] = key_value
#                             sample["instruction"] = encode_test_instruct("安居客", instruct, web_ele)
#                             print(sample)
#                             instruction_detail.append(sample)
#                             # c = input("!!!")
#     return instruction_detail


# print(all_ele())

In [ ]:
def all_ele():
    # web_ele = "网页[安居客]包括：可以[输入]名称为[小区名称]的[输入框]。可以[点击]名称为[不限 浦东 闵行 松江 宝山 嘉定 徐汇 青浦 静安 普陀 杨浦 奉贤 黄浦 虹口 长宁 金山 崇明 上海周边]的[" \
    #           "单选框]。可以[点击]名称为[1号线 2号线 3号线 4号线 5号线 6号线 7号线 8号线 9号线 10号线 11号线 12号线 13号线 14号线 15号线 16号线 17号线 18号线 磁悬浮 " \
    #           "浦江线]的[单选框]。可以[点击]名称为[100万以下 100-200万 200-250万 250-300万 300-350万 350-400万 400-500万 500-800万 " \
    #           "800-1000万]的[单选框]。可以[点击]名称为[不限 50㎡以下 50-60㎡ 60-70㎡ 70-80㎡ 80-90㎡ 90-100㎡ 100-120㎡ 120-140㎡ 140-210㎡ " \
    #           "210㎡以上]的[单选框]。可以[点击]名称为[房型]的[单选框]，元素为[不限 一室 二室 三室 四室 五室 五室以上]。可以[点击]名称为[特色]的单选框，元素为[近地铁 满五 满二]。可以[" \
    #           "点击]名称为[朝向]的[下拉框]，元素为[不限 东 南 西 北 南北 东西 东南 西南 东北 西北]。可以[点击]名称[楼层]的[复选框]元素为[不限 底层 低层 中层 高层 顶层]。可以[点击]名称[" \
    #           "年限]的下拉框，[下拉框选项]为[不限 2年内 2-5年 5-10年 10年以上]。可以[点击]名称[类型]的[下拉框]，[下拉框选项]为[不限 老公房 公寓 新里洋房 普通住宅 别墅 其它 " \
    #           "平房]。可以[点击]名称为[产权]的[下拉框]，[下拉框选项]为[不限 商品房住宅 商住两用 经济适用房 公房 动迁配套房 其他]。可以[点击]名称为[装修]的[下拉框]，[下拉框选项]为为[不限 毛坯 " \
    #           "普通装修 精装修 豪华装修]。 "
    advs = ["", "请", "帮助", "我要"]
    verbs = ["查找", "查询", "查找一下", "查询一下", "查一查"]

    # 输入框
    input_name = "汤臣一品 广洋院 王家堂小区".split()
    # 单选框
    regions = "不限 浦东 闵行 松江 宝山 嘉定 徐汇 青浦 静安 普陀 杨浦 奉贤 黄浦 虹口 长宁 金山 崇明 上海周边".split()
    region2s = "1号线 2号线 3号线 4号线 5号线 6号线 7号线 8号线 9号线 10号线 11号线 12号线 13号线 14号线 15号线 16号线 17号线 18号线 磁悬浮 浦江线".split()
    prices = "100万以下 100-200万 200-250万 250-300万 300-350万 350-400万 400-500万 500-800万 800-1000万".split()
    areas = "不限 50㎡以下 50-60㎡ 60-70㎡ 70-80㎡ 80-90㎡ 90-100㎡ 100-120㎡ 120-140㎡ 140-210㎡ 210㎡以上".split()
    house_classes = "不限 一室 二室 三室 四室 五室 五室以上".split()
    # 复选框
    characters = "近地铁 满五 满二".split()
    # 下拉框
    more1 = "不限 东 南 西 北 南北 东西 东南 西南 东北 西北".split()
    more2 = "不限 底层 低层 中层 高层 顶层".split()
    more3 = "不限 2年内 2-5年 5-10年 10年以上".split()
    more4 = "不限 老公房 公寓 新里洋房 普通住宅 别墅 其它 平房".split()
    more5 = "不限 商品房住宅 商住两用 经济适用房 公房 动迁配套房 其他".split()
    more6 = "不限 毛坯 普通装修 精装修 豪华装修".split()

    instruction_detail = []
    for adv in advs:
        for verb in verbs:
            for name in input_name:
                for price in prices:
                    for area in areas:
                        for house_class in house_classes:
                            sample = {}
                            key_value = {}
                            num_elements = random.randint(0, 3)
                            elements = random.choices(characters, k=num_elements)
                            character = ''.join(str(e) for e in elements)
                            instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + character + '的房子'
                            key_value['小区名字'] = {'dom_type': '输入框', 'value': name, 'action': '点击'}
                            key_value[price] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                            key_value[area] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                            key_value[house_class] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                            if len(elements) > 0:
                                for ele in elements:
                                    key_value[ele] = {'dom_type': '复选框', 'value': '', 'action': '点击'}
                            key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                            choser = random.randint(0, 7)
                            if choser == 1:
                                for more in more1:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + '朝向' + more+ character + '的房子'
                                    key_value['朝向'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            if choser == 2:
                                for more in more2:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + more+ character + '的房子'
                                    key_value['楼层'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            if choser == 3:
                                for more in more3:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + '房龄在' + more+ character + '的房子'
                                    key_value['房龄'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            if choser == 4:
                                for more in more4:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + more+ character + '的房子信息'
                                    key_value['物业类型'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            if choser == 5:
                                for more in more5:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + more+ character + '的房子信息'
                                    key_value['产权'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            if choser == 6:
                                for more in more6:
                                    instruct = adv + verb + name + '价格在' + price + '面积' + area + '户型' + house_class + more+ character + '的房子信息'
                                    key_value['毛坯'] = {"dom_type": "下拉框","value": "","action": "点击"}
                                    key_value[more] = {"dom_type": "下拉框选项","value": "","action": "点击"}
                            sample["key-value"] = key_value
                            sample["instruction"] = instruct
                            # sample["key-value"] = trans_key_values2web_element(key_value)
                            # sample["instruction"] = encode_train_instruct1(instruct, web_element_all)
                            instruction_detail.append(sample)
                            # c = input("!!!")
    return instruction_detail

instruction_detail=all_ele()
web_element_all_optimize = """[输入][小区名称][输入框]。[点击][不限 浦东 宝山 上海周边][单选框]。[点击][1号线 2号线 磁悬浮 浦江线][单选框]。[点击][100万以下 100-200万 400-500万][单选框]。[点击][不限 50㎡以下 50-60㎡ 60-70㎡ 210㎡以上][单选框]。[点击][不限 一室 五室以上][单选框]。[点击][近地铁 满五 满二][单选框]。[点击][朝向][下拉框]，[下拉框选项]为[不限 东 南 西北]。[点击][楼层][下拉框]，[下拉框选项]为[不限 底层 低层 顶层]。[点击][年限][下拉框]，[下拉框选项]为[不限 2年内 2-5年 10年以上]。[点击][类型][下拉框]，[下拉框选项]为[不限 老公房 平房]。[点击][产权][下拉框]，[下拉框选项]为[不限 商品房住宅 动迁配套房 其他]。[点击][装修][下拉框]，[下拉框选项]为[不限 毛坯 豪华装修]。"""
web_name = '安居客'
asset_anjuke_add = generate_train_dataset(label_trainset[14]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500,
                                         text_max_len=600)

In [ ]:
instruction_detail=all_ele()
web_element_all_optimize = """[输入][小区名称][输入框]。[点击][不限 浦东 宝山 上海周边][单选框]。[点击][1号线 2号线 磁悬浮 浦江线][单选框]。[点击][100万以下 100-200万 400-500万][单选框]。[点击][不限 50㎡以下 50-60㎡ 60-70㎡ 210㎡以上][单选框]。[点击][不限 一室 五室以上][单选框]。[点击][近地铁 满五 满二][单选框]。[点击][朝向][下拉框]，[下拉框选项]为[不限 东 南 西北]。[点击][楼层][下拉框]，[下拉框选项]为[不限 底层 低层 顶层]。[点击][年限][下拉框]，[下拉框选项]为[不限 2年内 2-5年 10年以上]。[点击][类型][下拉框]，[下拉框选项]为[不限 老公房 平房]。[点击][产权][下拉框]，[下拉框选项]为[不限 商品房住宅 动迁配套房 其他]。[点击][装修][下拉框]，[下拉框选项]为[不限 毛坯 豪华装修]。"""
web_name = '安居客'
asset_anjuke_add = generate_train_dataset(label_trainset[14]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500,
                                         text_max_len=600)

## 12306 GL

网页[火车票12306]包括:可以[点击]名称为[查询]的[按钮]。可以[点击]名称为[车票]的[按钮]。可以[点击]名称为[往返]的[按钮]。可以[点击]名称为[深沪京]的[按钮]。可以[点击]名称为[常用查询]的[按钮]。可以[点击]名称为[起售时间]的[按钮]。可以[输入]名称为[出发地]的[输入框]。可以[输入]名称为[到达地]的[输入框]。可以[输入]名称为[出发日期]的[输入框]。可以[输入]名称为[返程日期]的[输入框]。

In [ ]:
def random_datetime():
    import time
    import random
    a1 = (2020, 1, 1, 0, 0, 0, 0, 0, 0)  # 设置开始日期时间元组（1976-01-01 00：00：00）
    a2 = (2023, 12, 31, 23, 59, 59, 0, 0, 0)  # 设置结束日期时间元组（1990-12-31 23：59：59）
    start = time.mktime(a1)  # 生成开始时间戳
    end = time.mktime(a2)  # 生成结束时间戳
    t = random.randint(start, end)  # 在开始和结束时间戳中随机取出一个
    date_touple = time.localtime(t)  # 将时间戳生成时间元组
    date = time.strftime("%Y-%m-%d", date_touple)  # 将时间元组转成格式化字符串（1976-05-21）
    return date


def all_ele_train():
    # trainweb_ele = "网页[火车票12306]包括:可以[点击]名称为[查询]的[按钮]。可以[点击]名称为[车票]的[按钮]。可以[点击]名称为[往返]的[按钮]。可以[点击]名称为[深沪京]的[按钮]。可以[" \
    #                "点击]名称为[常用查询]的[按钮]。可以[输入]名称为[返程日期]的[输入框]。可以[输入]名称为[出发地]的[输入框]。可以[输入]名称为[到达地]的[输入框]。可以[输入]名称为[出发日期]的[" \
    #                "输入框]。可以[输入]名称为[返程日期]的[输入框]。 "
    advs = ['查询', '请查询', '查找', '请查找', '搜索', '请搜索', '检索', '请检索', '查一下', '请查一下', '检索一下', '请搜索：']
    # verbs = ["查找", "查询", "查找一下", "查询一下", "查一查", "搜索", "搜一下"]
    regions = ["北京", "上海", "广州", "深圳", "天津", "西安", "新疆", "哈尔滨", "拉萨", "太原"]
    regions_1 = ["北京", "上海", "广州", "深圳", "天津", "西安", "新疆", "哈尔滨", "拉萨", "太原"]
    # style = ["往返"]
    instruction_detail = []
    a = 100
    while a > 1:
        for adv in advs:
            # for verb in verbs:
                for region in regions:
                    sample = {}
                    key_value = {}
                    start_date = random_datetime()
                    instruct = adv + region + "车站在" + start_date + "起售的车票信息"
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    key_value['起售日期'] = {'dom_type': '日期输入框', 'value': start_date, 'action': '输入'}
                    key_value['常用查询'] = {'dom_type': '按钮', 'value':"", 'action': '点击'}
                    key_value['起售车站'] = {'dom_type': '输入框', 'value': region, 'action': '点击'}
                    key_value['起售时间'] = {'dom_type': '按钮', 'value': "", 'action': '点击'}
                    sample["key-value"] = key_value
                    sample["instruction"] = instruct
                    instruction_detail.append(sample)

        for adv in advs:
            # for verb in verbs:
                for region in regions:
                    sample = {}
                    key_value = {}
                    start_date = random_datetime()
                    region_1 = random.sample(regions_1, 1)[0]
                    instruct = adv + start_date + "的从" + region + "到" + region_1 + "的单程车票"
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    key_value["车票"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    key_value["单程"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    key_value['出发日期'] = {'dom_type': '日期输入框', 'value': start_date, 'action': '输入'}
                    key_value['出发地'] = {'dom_type': '输入框', 'value':region, 'action': '输入'}
                    key_value['到达地'] = {'dom_type': '输入框', 'value': region_1, 'action': '输入'}
                    sample["key-value"] = key_value
                    sample["instruction"] = instruct
                    instruction_detail.append(sample)

        for adv in advs:
        # for verb in verbs:
            for region in regions:
                sample = {}
                key_value = {}
                start_date = random_datetime()
                region_1 = random.sample(regions_1, 1)[0]
                instruct = adv + "出发地是" + region + "、到达地是" + region_1 + "的" + start_date + "的单程车票"
                key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value["车票"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value["单程"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value['出发日期'] = {'dom_type': '日期输入框', 'value': start_date, 'action': '输入'}
                key_value['出发地'] = {'dom_type': '输入框', 'value':region, 'action': '输入'}
                key_value['到达地'] = {'dom_type': '输入框', 'value': region_1, 'action': '输入'}
                sample["key-value"] = key_value
                sample["instruction"] = instruct
                instruction_detail.append(sample)

        for adv in advs:
        # for verb in verbs:
            for region in regions:
                sample = {}
                key_value = {}
                start_date = random_datetime()
                end_date = random_datetime()
                region_1 = random.sample(regions_1, 1)[0]
                instruct = adv + start_date + "出发、" + end_date + '返程的从' + region + '到' + region_1 + "的往返车票"
                key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value["车票"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value["返程"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                key_value['出发日期'] = {'dom_type': '日期输入框', 'value': start_date, 'action': '输入'}
                key_value['返程日期'] = {'dom_type': '日期输入框', 'value': end_date, 'action': '输入'}
                key_value['出发地'] = {'dom_type': '输入框', 'value':region, 'action': '输入'}
                key_value['到达地'] = {'dom_type': '输入框', 'value': region_1, 'action': '输入'}
                sample["key-value"] = key_value
                sample["instruction"] = instruct
                instruction_detail.append(sample)
        a -= 1
    return instruction_detail

instruction_detail = all_ele_train()
web_element_all_optimize = """[点击][查询][按钮]。[点击][车票][按钮]。[点击][常用查询][按钮]。[点击][订餐][按钮]。[点击][单程][按钮]。[点击][往返][按钮]。[点击][中转换成][按钮]。[点击][退改签][按钮]。[点击][正晚点][按钮]。[点击][检票口][按钮]。[点击][起售时间][按钮]。[点击][天气查询][按钮]。[输入][出发地][输入框]。[点击][预订][按钮]。[点击][餐饮订单][按钮]。[输入][到达地][输入框]。[输入][出发日期][日期输入框]。[输入][返程日期][日期输入框]。[输入][乘车日期][日期输入框]。[输入][开始日期][日期输入框]。[输入][结束日期][日期输入框]。[输入][关键字][输入框]。[点击][学生][单选框]。[点击][高铁/动车][单选框]。[点击][到达][单选框]。[点击][出发站][单选框]。[输入][车站][输入框]。[输入][车次][输入框]。"""
web_name = '火车票12306'
asset_12306_add = generate_train_dataset(label_trainset[16]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

In [ ]:
instruction_detail = all_ele_train()
web_element_all_optimize = """[点击][查询][按钮]。[点击][车票][按钮]。[点击][常用查询][按钮]。[点击][订餐][按钮]。[点击][单程][按钮]。[点击][往返][按钮]。[点击][中转换成][按钮]。[点击][退改签][按钮]。[点击][正晚点][按钮]。[点击][检票口][按钮]。[点击][起售时间][按钮]。[点击][天气查询][按钮]。[输入][出发地][输入框]。[点击][预订][按钮]。[点击][餐饮订单][按钮]。[输入][到达地][输入框]。[输入][出发日期][日期输入框]。[输入][返程日期][日期输入框]。[输入][乘车日期][日期输入框]。[输入][开始日期][日期输入框]。[输入][结束日期][日期输入框]。[输入][关键字][输入框]。[点击][学生][单选框]。[点击][高铁/动车][单选框]。[点击][到达][单选框]。[点击][出发站][单选框]。[输入][车站][输入框]。[输入][车次][输入框]。"""
web_name = '火车票12306'
asset_12306_add = generate_train_dataset(label_trainset[16]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

## 国家企业信用信息公示系统 GL

网页[国家企业信用信息公示系统]包括:可以[点击]名称为[搜索]的[按钮]。可以[点击]名称为[严重违法失信名单]的[单选框]。可以[点击]名称为[经营异常名录]的[单选框]。可以[点击]名称为[无动产抵押登记信息]的[单选框]。可以[点击]名称为[地区]的[单选框]，[单选框]元素为[浙江 福建 天津]等其他地名。可以[点击]名称为[无行政许可信息]的[单选框]。可以[点击]名称为[有动产抵押登记信息]的[单选框]。可以[点击]名称为[吊销，已注销]的[单选框]。可以[点击]名称为[成立15年以上]的[单选框]。可以[点击]名称为[成立1-5年]的[单选框]。可以[点击]名称为[无商标注册信息]的[单选框]。可以[点击]名称为[有行政处罚信息]的[单选框]。可以[点击]名称为[无行政处罚信息]的[单选框]

In [ ]:
# 国家企业信用信息公示系统
def nation_company_info():
    advs = ["", "请", "帮助", "我要"]
    verbs = ["查找", "查询", "查找一下", "查询一下", "查一查", "搜索", "搜一下"]
    regions = ["北京 ", "上海", "广州", "深圳", "天津", "西安", "新疆"]
    status = ["成立15年以上", "成立10-15年", "成立5-10年", "成立1-5年", "成立1年内"]
    status2 = ["已注销", "未注销"]
    status3 = ["有商标注册", "无商标注册"]
    status4 = ["注销", "存续（在营、开业、在册）"]
    endstatus = ["经营异常名录", "企业信息", "严重违法失信名单"]
    instruction_detail = []
    a=100
    while a>0:
        for adv in advs:
            for verb in verbs:
                for endstatu in endstatus:
                    sample = {}
                    key_value = {}
                    num_elements = random.randint(0, 1)
                    region = random.choices(regions, k=num_elements)
                    num_elements2 = random.randint(0, 1)
                    statu = random.choices(status, k=num_elements2)
                    num_elements3 = random.randint(0, 1)
                    statu2 = random.choices(status2, k=num_elements3)
                    num_elements4 = random.randint(0, 1)
                    statu3 = random.choices(status3, k=num_elements4)
                    num_elements5 = random.randint(0, 1)
                    statu4 = random.choices(status4, k=num_elements5)
                    instruct = adv + verb + str(region) + str(statu) + str(statu2) + str(statu3) + str(statu4) + "的" + str(endstatu)
                    instruct = instruct.replace("['","").replace("']","").replace("[]","")
                    if num_elements != 0:
                        key_value[region[0]] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    if num_elements2 != 0:
                        key_value[statu[0]] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    if num_elements3 != 0:
                        key_value[str(statu2)] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    if num_elements4 != 0:
                        key_value[statu3[0]] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    if num_elements5 != 0:
                        key_value[statu4[0]] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    key_value["搜索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    sample["key-value"] = key_value
                    sample["instruction"] = instruct
                    instruction_detail.append(sample)
        a -= 1
    return instruction_detail

instruction_detail = nation_company_info()
web_element_all_optimize = """[点击][搜索][按钮]。[点击][严重违法失信名单][单选框]。[点击][经营异常名录][单选框]。[点击][无动产抵押登记信息][单选框]。[点击][[浙江 福建 天津][单选框]。[点击][无行政许可信息][单选框]。[点击][有动产抵押登记信息][单选框]。[点击][吊销，已注销][单选框]。[点击][成立15年以上][单选框]。[点击][成立1-5年][单选框]。[点击][无商标注册信息][单选框]。[点击][有行政处罚信息][单选框]。[点击][无行政处罚信息][单选框]。"""
web_name = '国家企业信用信息公示系统'
asset_guojiaqiye_add = generate_train_dataset(label_trainset[11]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

In [ ]:
instruction_detail = nation_company_info()
web_element_all_optimize = """[点击][搜索][按钮]。[点击][严重违法失信名单][单选框]。[点击][经营异常名录][单选框]。[点击][无动产抵押登记信息][单选框]。[点击][[浙江 福建 天津][单选框]。[点击][无行政许可信息][单选框]。[点击][有动产抵押登记信息][单选框]。[点击][吊销，已注销][单选框]。[点击][成立15年以上][单选框]。[点击][成立1-5年][单选框]。[点击][无商标注册信息][单选框]。[点击][有行政处罚信息][单选框]。[点击][无行政处罚信息][单选框]。"""
web_name = '国家企业信用信息公示系统'
asset_guojiaqiye_add = generate_train_dataset(label_trainset[11]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=1000,
                                         rule_num2=500,
                                         rule_num3=1000,
                                         common_num=250,
                                         text_max_len=600)

## 国家统计局  GL

网页[国家统计局]包括:可以[点击]名称为[开始搜索]的[按钮]。可以[输入]名称为[包含以下全部的关键词]的[输入框]。可以[输入]名称为[包含以下的完整关键词]的[输入框]。可以[输入]名称为[包含以下任意一个关键词]的[输入框]。可以[输入]名称为[不包含以下关键词]的[输入框]。可以[输入]名称为[自定义时间1]的[输入框]。可以[输入]名称为[自定义时间2]的[输入框],可以[点击]名称为[文档格式]的[单选框],[单选框]元素为[全部格式WORD PPT EXCEL PDF TXT ZIP]。可以[点击]名称为[时间范围]的[单选框],[单选框]元素为[全部时间 一年内 一月内 一周内 自定义]。

In [ ]:
def nation_statics():
    advs = ["", "请", "帮助", "我要"]
    verbs = ["查找", "查询", "查找一下", "查询一下", "查一查", "搜索", "搜一下"]
    search_class = ["包含以下全部的关键词:", "包含以下的完整关键词:", "包含以下任意一个关键词:", "不包含以下关键词:"]
    file_style = ["全部格式", "WORD", "PPT", "EXCEL", "PDF", "TXT", "ZIP"]
    time_style = ["全部时间", "一年内", "一月内", "一周内", "自定义"]
    nums = ["5", "10", "20"]
    排序方式 = ["按相关度排序", "按时间排序"]
    搜索位置 = ["任何地方", "仅标题"]
    some = ["职务评聘", "统计年鉴、县域", "直辖市", "义务教育", "地质勘查、全国", "失信企业信息"]
    instruction_detail = []
    a = 10
    while a>0:
        for adv in advs:
            for verb in verbs:
                for _ in range(200):
                    sample = {}
                    key_value = {}
                    result = random.choices(some, k=4)
                    time_style1 = random.choices(time_style, k=1)
                    key_value[search_class[0]] = {'dom_type': '输入框', 'value': result[0], 'action': '点击'}
                    key_value[search_class[1]] = {'dom_type': '输入框', 'value': result[1], 'action': '点击'}
                    key_value[search_class[2]] = {'dom_type': '输入框', 'value': result[2], 'action': '点击'}
                    key_value[search_class[3]] = {'dom_type': '输入框', 'value': result[3], 'action': '点击'}
                    if time_style1 == "自定义":
                        time1 = random_datetime()
                        time2 = random_datetime()
                        instruct = adv + verb + search_class[0] + result[0] + search_class[1] + result[1] + search_class[
                            2] + result[2] + search_class[3] + "时间在:" + time1 + "-" + time2 + result[3] + "的结果"
                        key_value["自定义时间1"] = {'dom_type': '输入框', 'value': time1, 'action': '点击'}
                        key_value["自定义时间2"] = {'dom_type': '输入框', 'value': time2, 'action': '点击'}
                    else:
                        instruct = adv + verb + str(search_class[0]) + str(result[0]) + str(search_class[1]) + str(result[1]) + \
                                   str(search_class[2]) + str(result[2]) + str(search_class[3]) + "时间在:" + str(time_style1) + \
                                   str(result[3]) + "的结果"
                        key_value[time_style1[0]] = {'dom_type': '单选框', 'value': '', 'action': '点击'}
                    key_value["开始搜索"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    sample["key-value"] = key_value
                    sample["instruction"] = instruct
                    instruction_detail.append(sample)
        a -= 1
    return instruction_detail

instruction_detail = nation_statics()
web_element_all_optimize = """[点击][开始搜索][按钮]。[输入][包含以下全部的关键词][输入框]。[输入][不包含以下的完整关键词][输入框]。[输入][包含以下任意一个关键词][输入框]。[输入][不包含以下关键词][输入框]。[输入][自定义时间_1][输入框]。[输入][自定义时间_2][输入框],[点击][全部格式 WORD PPT EXCEL PDF TXT ZIP][单选框]。[点击][全部时间 一年内 一月内 一周内 自定义][单选框]。[点击][按相关度排序 按时间排序][单选框]。[点击][任何地方 仅标题][单选框]。[点击][显示条数][下拉框]，[下拉框选项]为[5 10 20]。"""
web_name = '国家企业信用信息公示系统'
asset_gjtjj_add = generate_train_dataset(label_trainset[12]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500,
                                         text_max_len=600)

In [ ]:
instruction_detail = nation_statics()
web_element_all_optimize = """[点击][开始搜索][按钮]。[输入][包含以下全部的关键词][输入框]。[输入][不包含以下的完整关键词][输入框]。[输入][包含以下任意一个关键词][输入框]。[输入][不包含以下关键词][输入框]。[输入][自定义时间_1][输入框]。[输入][自定义时间_2][输入框],[点击][全部格式 WORD PPT EXCEL PDF TXT ZIP][单选框]。[点击][全部时间 一年内 一月内 一周内 自定义][单选框]。[点击][按相关度排序 按时间排序][单选框]。[点击][任何地方 仅标题][单选框]。[点击][显示条数][下拉框]，[下拉框选项]为[5 10 20]。"""
web_name = '国家企业信用信息公示系统'
asset_gjtjj_add = generate_train_dataset(label_trainset[12]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                         rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500,
                                         text_max_len=600)

## 交通安全综合服务管理平台 GL

网页[交通安全综合服务管理平台]包括:可以[点击]名称为[搜索]的[按钮]。可以[输入]名称为[决定书编号]的[输入框]。可以[输入]名称为[企业名称]的[输入框]。可以[输入]名称为[号牌号码]的[输入框]。可以[输入]名称为[社会信用代码]的[输入框]。可以[输入]名称为[姓名]的[输入框]。可以[输入]名称为[公示时间_1]的[输入框]。可以[输入]名称为[公示时间_2]的[输入框]。

## 上交所 GL

网页[上交所]包括:可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[证券代码或简称]的[输入框]。可以[点击]名称为[板块]的[下拉框]，其[下拉框选项]为[主板A股 主板B股 科创板]。可以[点击]名称为[地区]的[下拉框]，其[下拉框选项]为[安徽 北京 黑龙江 湖北 湖南 吉林 江苏 江西 辽宁 内蒙古 宁夏 青海 福建 山东 山西 陕西 上海 四川 天津 西藏 新疆 云南 浙江 甘肃 重庆 广东 广西 贵州 海南 河北 河南]。可以[点击]名称为[行业]的[下拉框]，其[下拉框选项]为[A.农、林、牧、渔业 B.采掘业 C.制造业 D.电力、热力、燃气及水生产和供应业 E.建筑业 F.批发和零售业 G.交通运输、仓储和邮政业 H.住宿和餐饮业 I.信息传输、软件和信息技术服务业 J.金融业 K.房地产业 L.租赁和商务服务业 M.科学研究和技术服务业 N.水利、环境和公共设施管理业 O.居民服务、修理和其他服务业 P.教育 Q.卫生和社会工作 R.文化、体育和娱乐业 S.综合 高端装备 节能环保 生物医药 新材料 新能源 新一代信息技术]。

In [ ]:
import json
import random

page_source = '上交所'
instructions = []
# codes = []
blocks = []
locations = []
industries = []
for i, instruction in enumerate(instruction_trainset[1]['instruction_detail']):
    instruction_str = instruction['instruction']
    instructions.append(instruction_str)
    message = instruction['key-value']
    try:
        block = message["板块"]
        blocks.append(block)
    except:
        print("这个实例没有板块")
    try:
        location = message['地区']
        locations.append(location)
    except:
        print("这个实例没有地区")

locations = list(set(locations))
blocks = list(set(blocks))

index_industries = ['P.教育', 'S.综合',
 'L.租赁和商务服务业',
 'G.交通运输、仓储和邮政业',
 'I.信息传输、软件和信息技术服务业',
 'C.制造业',
 'D.电力、热力、燃气及水生产和供应业',
 'B.采掘业',
 'J.金融业',
 'K.房地产业',
 'R.文化、体育和娱乐业',
 'A.农、林、牧、渔业',
 'F.批发和零售业',
 'O.居民服务、修理和其他服务业',
 'M.科学研究和技术服务业',
 'Q.卫生和社会工作',
 'N.水利、环境和公共设施管理业',
 'E.建筑业',
 'H.住宿和餐饮业']
industries_1 = ['高端装备', '节能环保', '生物医药', '新材料', '新能源', '新一代信息技术']
industries = [x.split('.')[1] for x in index_industries]
industries_dic = {}
for i in range(len(industries)):
    industries_dic[industries[i]] = index_industries[i]
codes = ['600006', '600000', '600010', '600004', '600008', '600009', '600007']
names = ['包钢股份', '浦发银行', '中国国贸', '首创环保', '东风汽车', '白云机场', '上海机场']


querys_head = ['查询', '请查询', '查找', '请查找', '搜索', '请搜索', '检索', '请检索', '查一下', '请查一下', '检索一下', '请搜索：']

instruction_detail_1 = []
for query in querys_head:
            for block in blocks:
                    # situation 1
                    sample = {}
                    instruct = query + block + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['板块']
                    for action in actions:
                        key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                        key_value[block] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                    sample["key-value"] = key_value
                    instruction_detail_1.append(sample)

instruction_detail_2 = []
for query in querys_head:
            for name in names:
                    # situation 2
                    sample = {}
                    instruct = query + name + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['证券代码或简称']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_2.append(sample)

instruction_detail_3 = []
for query in querys_head:
            for code in codes:
                    # situation 3
                    sample = {}
                    instruct = query + '证券代码' + code + '的信息'
                    sample["instruction"] = instruct
                    key_value = {}
                    key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                    actions = ['证券代码或简称']
                    for action in actions:
                        key_value[action] = {'dom_type': '输入框', 'value': code, 'action': '输入'}
                    sample["key-value"] = key_value
                    instruction_detail_3.append(sample)

instruction_detail_4 = []
for query in querys_head:
            for location in locations:
                for ind in industries_1:
                    for name in names:
                        # situation 4
                        sample = {}
                        instruct = query + location + ind + '行业科创板的' + name + '的信息'
                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['证券代码或简称', '板块', '地区', '行业']
                        for action in actions:
                            if action=='证券代码或简称':
                                key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                            elif action=='板块':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value['科创板'] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='地区':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='行业':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[ind] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        instruction_detail_4.append(sample)

instruction_detail_5 = []
for query in querys_head:
            for location in locations:
                for ind in industries_1:
                    for name in names:
                        # situation 5
                        sample = {}
                        instruct = query + location + name + '的' + ind  + '行业科创板的信息'
                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['证券代码或简称', '板块', '地区', '行业']
                        for action in actions:
                            if action=='证券代码或简称':
                                key_value[action] = {'dom_type': '输入框', 'value': name, 'action': '输入'}
                            elif action=='板块':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value['科创板'] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='地区':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='行业':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[ind] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        instruction_detail_5.append(sample)

instruction_detail_6 = []
for query in querys_head:
            for location in locations:
                for ind in industries:
                    for block in ['主板A股', '主板B股']:
                        # situation 6
                        sample = {}
                        #  '请检索福建农、林、牧、渔业的主板B股的信息'
                        instruct = query + location + ind + '的' + block  + '的信息'
                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['板块', '地区', '行业']
                        for action in actions:
                            if action=='板块':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[block] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='地区':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='行业':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[industries_dic[ind]] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        instruction_detail_6.append(sample)

instruction_detail_7 = []
for query in querys_head:
            for location in locations:
                for ind in industries_1:
                        # situation 7
                        sample = {}
                        #  '查一下安徽新材料行业科创板的信息'
                        instruct = query + location + ind  + '行业科创板的信息'
                        sample["instruction"] = instruct
                        key_value = {}
                        key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                        actions = ['板块', '地区', '行业']
                        for action in actions:
                            if action=='板块':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value['科创板'] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='地区':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            elif action=='行业':
                                key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                key_value[ind] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                        sample["key-value"] = key_value
                        instruction_detail_7.append(sample)

instruction_detail_8 = []
for query in querys_head:
            for location in locations:
                for ind in industries:
                    for block in ['主板A股', '主板B股']:
                        for code in codes:
                            # situation 8
                            sample = {}
                            #查找出辽宁金融业的主板A股的证券代码是600006的信息
                            instruct = query + location + ind + '的' + block + '的证券代码是' + code + '的信息'
                            sample["instruction"] = instruct
                            key_value = {}
                            key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                            actions = ['证券代码或简称', '板块', '地区', '行业']
                            for action in actions:
                                if action=='证券代码或简称':
                                    key_value[action] = {'dom_type': '输入框', 'value': code, 'action': '输入'}
                                elif action=='板块':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value[block] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                                elif action=='地区':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                                elif action=='行业':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value[industries_dic[ind]] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            sample["key-value"] = key_value
                            instruction_detail_8.append(sample)

instruction_detail_9 = []
for query in querys_head:
            for location in locations:
                for ind in industries_1:
                        for code in codes:
                            # situation 9
                            sample = {}
                            #查找湖北节能环保行业科创板的证券代码是600009的信息
                            instruct = query + location + ind + '行业科创板' + '的证券代码是' + code + '的信息'
                            sample["instruction"] = instruct
                            key_value = {}
                            key_value["查询"] = {'dom_type': '按钮', 'value': '', 'action': '点击'}
                            actions = ['证券代码或简称', '板块', '地区', '行业']
                            for action in actions:
                                if action=='证券代码或简称':
                                    key_value[action] = {'dom_type': '输入框', 'value': code, 'action': '输入'}
                                elif action=='板块':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value['科创板'] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                                elif action=='地区':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value[location] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                                elif action=='行业':
                                    key_value[action] = {'dom_type': '下拉框', 'value': '', 'action': '点击'}
                                    key_value[ind] = {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}
                            sample["key-value"] = key_value
                            instruction_detail_9.append(sample)
random.shuffle(instruction_detail_1)
random.shuffle(instruction_detail_2)
random.shuffle(instruction_detail_3)
random.shuffle(instruction_detail_4)
random.shuffle(instruction_detail_5)
random.shuffle(instruction_detail_6)
random.shuffle(instruction_detail_7)
random.shuffle(instruction_detail_8)
random.shuffle(instruction_detail_9)
instruction_detail = []
instruction_detail.extend(instruction_detail_1[:5000])
instruction_detail.extend(instruction_detail_2[:5000])
instruction_detail.extend(instruction_detail_3[:5000])
instruction_detail.extend(instruction_detail_4[:5000])
instruction_detail.extend(instruction_detail_5[:5000])
instruction_detail.extend(instruction_detail_6[:5000])
instruction_detail.extend(instruction_detail_7[:5000])
instruction_detail.extend(instruction_detail_8[:5000])
instruction_detail.extend(instruction_detail_9[:5000])

print(len(instruction_detail))
random.shuffle(instruction_detail)
print("generate all num: %d" % len(instruction_detail))
instruction_detail = random.sample(instruction_detail, 10000)
print(len(instruction_detail))

web_element_all = """可以[点击]名称为[查询]的[按钮]。可以[输入]名称为[证券代码或简称]的[输入框]。可以[点击]名称为[板块]的[下拉框]，其[下拉框选项]为[主板A股 主板B股 科创板]。可以[点击]名称为[地区]的[下拉框]，其[下拉框选项]为[安徽 北京 黑龙江 湖北 湖南 吉林 江苏 江西 辽宁 内蒙古 宁夏 青海 福建 山东 山西 陕西 上海 四川 天津 西藏 新疆 云南 浙江 甘肃 重庆 广东 广西 贵州 海南 河北 河南]。可以[点击]名称为[行业]的[下拉框]，其[下拉框选项]为[A.农、林、牧、渔业 B.采掘业 C.制造业 D.电力、热力、燃气及水生产和供应业 E.建筑业 F.批发和零售业 G.交通运输、仓储和邮政业 H.住宿和餐饮业 I.信息传输、软件和信息技术服务业 J.金融业 K.房地产业 L.租赁和商务服务业 M.科学研究和技术服务业 N.水利、环境和公共设施管理业 O.居民服务、修理和其他服务业 P.教育 Q.卫生和社会工作 R.文化、体育和娱乐业 S.综合 高端装备 节能环保 生物医药 新材料 新能源 新一代信息技术]。"""
web_element_all_optimize = """[点击][查询][按钮]。[输入][证券代码或简称][输入框]。[点击][板块][下拉框]，其[下拉框选项]为[主板A股 主板B股 科创板]。[点击][地区][下拉框]，其[下拉框选项]为[安徽 北京 黑龙江 湖北 湖南 吉林 江苏 江西 辽宁 内蒙古 宁夏 青海 福建 山东 山西 陕西 上海 四川 天津 西藏 新疆 云南 浙江 甘肃 重庆 广东 广西 贵州 海南 河北 河南]。[点击][行业][下拉框]，其[下拉框选项]为[A.农、林、牧、渔业 B.采掘业 C.制造业 D.电力、热力、燃气及水生产和供应业 E.建筑业 F.批发和零售业 G.交通运输、仓储和邮政业 H.住宿和餐饮业 I.信息传输、软件和信息技术服务业 J.金融业 K.房地产业 L.租赁和商务服务业 M.科学研究和技术服务业 N.水利、环境和公共设施管理业 O.居民服务、修理和其他服务业 P.教育 Q.卫生和社会工作 R.文化、体育和娱乐业 S.综合 高端装备 节能环保 生物医药 新材料 新能源 新一代信息技术]。"""
web_name = '上交所'
asset_sjs_add = generate_train_dataset(label_trainset[1]['instruction_detail'],
                                         instruction_detail,
                                         web_name,
                                         web_element_all_optimize,
                                          rule_num1=2000,
                                         rule_num2=1000,
                                         rule_num3=2000,
                                         common_num=500,
                                         text_max_len=600)

In [ ]:
words = "[输入][小区名称][输入框]。[点击][不限 浦东 宝山 上海周边][单选框]。[点击][1号线 2号线 磁悬浮 浦江线][单选框]。[点击][100万以下 100-200万 400-500万][单选框]。[点击][不限 50㎡以下 50-60㎡ 60-70㎡ 210㎡以上][单选框]。[点击][房型][单选框]，元素为[不限 一室 五室以上]。[点击][特色]的单选框，元素为[近地铁 满五 满二]。[点击][朝向][下拉框]，元素为[不限 东 南 西北]。[点击]名称[楼层][复选框]元素为[不限 底层 低层 顶层]。[点击]名称[年限]的下拉框，[下拉框选项]为[不限 2年内 2-5年 10年以上]。[点击]名称[类型][下拉框]，[下拉框选项]为[不限 老公房 平房]。[点击][产权][下拉框]，[下拉框选项]为[不限 商品房住宅 动迁配套房 其他]。[点击][装修][下拉框]，[下拉框选项]为为[不限 毛坯 豪华装修]。[输入][小区名称][输入框]。[点击][不限 浦东 宝山 上海周边][单选框]。[点击][1号线 2号线 磁悬浮 浦江线][单选框]。[点击][100万以下 100-200万 400-500万][单选框]。[点击][不限 50㎡以下 50-60㎡ 60-70㎡ 210㎡以上][单选框]。[点击][房型][单选框]，元素为[不限 一室 五室以上]。[点击][特色]的单选框，元素为[近地铁 满五 满二]。[点击][朝向][下拉框]，元素为[不限 东 南 西北]。[点击]名称[楼层][复选框]元素为[不限 底层 低层 顶层]。[点击]名称[年限]的下拉框，[下拉框选项]为[不限 2年内 2-5年 10年以上]。[点击]名称[类型][下拉框]，[下拉框选项]为[不限 老公房 平房]。[点击][产权][下拉框]，[下拉框选项]为[不限 商品房住宅 动迁配套房 其他]。[点击][装修][下拉框]，[下拉框选项]为为[不限 毛坯 豪华装修]。"
print(len(words))

842
